**LATAM GMV / MRR at OPP (Client/Location) Level**

20 sep. 2022

Script to pull GMV and MRR data for all LATAM customers - per opportunity - in a monthly basis.

In [1]:
# Import packages, libraries and modules

#!pip install ds
import ds
import warnings
warnings.simplefilter("ignore")
import json
import os
import pandas as pd
import numpy as np
import re
import datetime
from datetime import timedelta
from ds.utilities.io import ds_trino
import numpy as np

# Display settings
from IPython.display import HTML, display
display(
    HTML(
        data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 95%; }
    div#maintoolbar-container { width: 95%; }
</style>
"""
    )
)

# Set max rows and columns to display
pd.options.display.max_columns = None
pd.options.display.max_rows = 100
pd.options.mode.chained_assignment = None

In [2]:
# Create connection to SuperSet via ds_trino
conn = ds_trino.create_trino_connection()

**Querying GMV data from superset**

- Data per opportunity per month
- Last 12 months

In [3]:
opty_query = """
     
with ordenes as (



with scan_out as ( SELECT
      CAST(order_uuid AS VARCHAR) AS order_id
      ,TRUE AS order_scanned_out
      ,count ((order_uuid)) as scanned_items
    FROM
     
      hudi_ingest.inventory_restaurant_platform_cockroach_dbreplica.inventory_transaction 
    WHERE
      day_partition>= DATE_FORMAT(DATE_TRUNC('month', current_date) - interval '12' month, '%Y-%m-%d')
      
      AND type = 'TRANSACTION_ORDER_SCANOUT'
      AND cause = 'CAUSE_MANUAL'
    GROUP BY
      1,2)



SELECT 
DATE(from_unixtime(reference_time/1000, facility_timezone)) AS created_date,
--que empiece en domingo
date_trunc('month', DATE(from_unixtime(reference_time/1000, facility_timezone))) as created_month,
date_trunc('week', DATE(from_unixtime((reference_time)/1000, facility_timezone))+ interval '1' day)- interval '1' day as created_week,
FLOOR((DATE_DIFF('day', DATE(created_at_ts), DATE(AT_TIMEZONE(current_timestamp, facility_timezone))) - 3) / 7) + 1 AS order_weeks_back_local,
cast(local_created_at_ts as time) as local_created_time,
facility_timezone,
'LATAM' as region,
dc.country,



CASE 

when dc.country='Brazil' then 'Brazil'

else 'SSL'

end as subregion,


(
CASE
   WHEN
      facility_id IN 
      (
         'd0c82479-ad1c-4a7e-aaa5-d83ee49e9df7',  -- LIM - Surquillo
         '6d2d117b-3fa9-4a2c-85b5-9b86a758f849'   -- LIM - Magdalena
      )
   THEN
      'Lima'
   WHEN
      facility_id IN 
      (
         'd65136fe-862a-4da5-81ef-580c28f8c61a'  -- UIO - 10 de Agosto
      )
   THEN
      'Quito'
   WHEN
      facility_id IN 
      (
         '567933f5-72a7-4b7a-b6b0-9e1ff87f5bcf', -- BOG - Prado
         '199c5433-d863-4d5e-bb6e-a53d91e0605f', -- Modelia  BOGOTA
         '4d78d1e1-5bce-4307-8175-51c816120fce', -- BOG - Castellana
         'b2f57126-8e64-4b11-964f-c497a3b53571', -- BOG - Cocinas de la 73
         'a5852445-e223-46c9-be99-ba33f204bcd1', -- BOG - Puente Aranda
         '76a6ef30-ddb2-4178-8c37-86c0d10db4b3', -- BOG - Atabanza
         'dd16bf0e-8a89-4f21-bc02-1c9793227502', -- BOG - San Felipe
         '3a6e7382-f2c0-4ebc-8f12-61f4d44dee82',  -- BOG - Cedritos
         '757182ff-7a54-4571-91d4-0027253c0117' --Salitre
      )
   THEN
      'Bogota'
  
  WHEN
      facility_id in 
      ('0016d9fc-4f27-4a05-88cb-90cd1cdf311f' --demoda
      )
  THEN 
      'Medellin'
      
      
      
   WHEN
      facility_id in 
      (
         '46fb5e19-3a79-4eda-b119-dee3a331fd81'  -- CLO - Chipichape
      )
   THEN
      'Cali' 
   WHEN
      facility_id in 
      (
         '58b818c6-615e-4316-b4a3-b9a6bb914c30'  --BAQ - Alto Prado
      )
   THEN
      'Barranquilla' 
   WHEN
      facility_id in 
      (
         '9eede066-b7e7-43c4-907c-21ece1688b14',  -- SJO - Guadalupe
         '967f0a2e-9c8e-4b35-929b-37420d9439c9'   -- SJO - Pavas
      )
   THEN
      'San Jose - CR' 
   WHEN
      facility_id in 
      (
         '47e2d054-8b82-420c-b1b0-18010f57489c', -- MEX - Insurgentes Sur
         '565273fe-0074-421d-b1e1-796d1e4390e1', -- MEX - Napoles
         '864e0a3c-ecba-4305-b3ff-f22edd58c902', -- MEX - Polanco
         'bcb04949-3cda-4ca0-be39-b9f579e356ca', -- MEX - Roma
         '9101d4f9-828f-45c5-b8b6-16986e76b374', -- MEX - Pedregal
         'b9545910-bca6-4b2a-a603-363721e7a23d', -- MEX - Santa Fe
         'da27f06f-01fd-4106-88d9-b3fd41446515'  -- MEX - Lindavista
      )
   then
      'Mexico City'
   when
      facility_id in 
      (
         '394c0103-1c5f-424b-b398-68455fced5bb', -- MTY - Cumbres
         '6d2d6c66-47aa-4f6e-ad75-58f68cc36c5f', -- MTY - Leones NK
         '988c22e5-20c2-42a5-bc30-989c68ff5eee', -- MTY - Tec
         '1b90b4ae-6d8f-4a73-bb9e-708c83387430', -- MTY - Santa Maria
         '88774bba-2e34-4771-8e64-8f059adf02a3'  -- MTY - Valle Oriente
      )
   then
      'Monterrey'
   when
      facility_id in 
      (
         'a9a6c4be-32b7-47cf-b783-7a5227a89ed4',  -- GDL - Ciudad del Sol
         'c66ce711-5486-433f-a9b7-1b36ae3a6dee' -- GDL - av mexico
      )
   then
      'Guadalajara'
   WHEN
      facility_id IN
      (
         '7b5c774f-7739-4d8e-a392-d49547d45fc3', -- BHZ - Contagem Industrial
         '554263c6-bde4-4d4c-92a7-21ed77e122cd', -- BHZ - Paracatu
         '522c745a-fa86-42de-83ca-227cbd3cb353', -- BHZ - Arthur Guimaraes
         '4518d99c-fa47-4da7-b5f5-c071c78b5ac2'  -- BHZ - Tancredo Neves'
      )
   then
      'Belo Horizonte' 
   WHEN
      facility_id IN
      (
         '81e6538d-c45b-4479-92e8-fc4e04373e12', -- RIO - Jose Bonifacio
         '466abb82-57b9-4e0d-9c5b-d1ef6c7f92bb', -- RIO - Barra
         '0ee5dccf-ed5b-465c-9858-b53351353e3b',  -- RIO - Camerino
         '976ae838-18e0-40bb-a8dc-360353c01b6d', -- RIO - Niteroi
         '3759ad21-4537-48ea-b1f1-48809e8a03ac', --rio botafogo
         'f7a65e9f-3342-48d2-a0fb-e9e4633c4bf6' -- RIO - Tijuca
      )
   then
      'Rio de Janeiro' 
   WHEN
      facility_id IN
      (
         '6c601837-403c-425f-a51c-31428ed1a67e', -- SAO - Rua do Acre
         '87a40094-cf48-4032-8166-e4cee61bce2f', -- SAO - Dino
         '4326199f-628a-4b00-a6e6-054fb5182c64', -- SAO - Guararapes (Brooklin)
         'dcc461ae-c80c-46cc-a5dc-4a5a72761510', -- SAO - Lapa Clelia
         '7aae3115-311b-449f-88f4-0f1612ecfcee', -- SAO - Borba
         '1840180b-9a3b-4fb6-b35c-ef4e6785b26e', -- SAO - Abelardo
         '88aaaf99-a444-4e0f-99f6-267b635faa53',  -- SAO - Vargas
         'a63559d1-93c6-4f60-b683-1e207ccda10a', -- SAO - Batista
          --'89cf86b9-9e74-4657-86df-2a851492af88', -- SAO - Guarulhos
          '89cf86b9-9e74-4657-86df-2a851492af88', --sao espanhola
          '14d9bb92-6376-4d69-adb6-64f97e4b3561', -- sao ferreira
          '812e6913-d78e-4dbc-877d-16993853aa7e' -- sao domingos
      )
   then
      'Sao Paulo'
      
  WHEN 
        facility_id IN 
        (
        'c559b06b-e193-495c-85cd-a5d1608ca27d'  -- BSB - Aguas Claras
        )
    THEN
        'Brasilia'
      
      
    WHEN
        facility_id IN
        (
            'db3a588c-59ac-458e-84de-9e1c8ef5e3e8' -- 'SCL - Recoleta'
        )
    THEN
        'Santiago'
END
) AS city,
(
CASE
   WHEN facility_id = 'd0c82479-ad1c-4a7e-aaa5-d83ee49e9df7' THEN 'LIM - Surquillo'
   WHEN facility_id = '6d2d117b-3fa9-4a2c-85b5-9b86a758f849' THEN 'LIM - Magdalena'
   WHEN facility_id = 'd65136fe-862a-4da5-81ef-580c28f8c61a' THEN 'UIO - 10 de Agosto'
   WHEN facility_id = '567933f5-72a7-4b7a-b6b0-9e1ff87f5bcf' THEN 'BOG - Prado'
   WHEN facility_id = '4d78d1e1-5bce-4307-8175-51c816120fce' THEN 'BOG - Castellana'
   WHEN facility_id = 'b2f57126-8e64-4b11-964f-c497a3b53571' THEN 'BOG - Cocinas de la 73'
   WHEN facility_id = 'a5852445-e223-46c9-be99-ba33f204bcd1' THEN 'BOG - Puente Aranda'
   WHEN facility_id = '76a6ef30-ddb2-4178-8c37-86c0d10db4b3' THEN 'BOG - Atabanza'
   WHEN facility_id = 'dd16bf0e-8a89-4f21-bc02-1c9793227502' THEN 'BOG - San Felipe'
   WHEN facility_id = '3a6e7382-f2c0-4ebc-8f12-61f4d44dee82' THEN 'BOG - Cedritos'
   when facility_id = '757182ff-7a54-4571-91d4-0027253c0117' Then 'BOG - Salitre 2 EM'
   when facility_id = '0016d9fc-4f27-4a05-88cb-90cd1cdf311f' THEN 'MED - DModa'
 
   
   WHEN facility_id = '46fb5e19-3a79-4eda-b119-dee3a331fd81' THEN 'CLO - Chipichape' 
   WHEN facility_id = '58b818c6-615e-4316-b4a3-b9a6bb914c30' THEN 'BAQ - Alto Prado' 
   WHEN facility_id = '9eede066-b7e7-43c4-907c-21ece1688b14' THEN 'SJO - Guadalupe' 
   WHEN facility_id = '967f0a2e-9c8e-4b35-929b-37420d9439c9' THEN 'SJO - Pavas'
   
   WHEN facility_id = '47e2d054-8b82-420c-b1b0-18010f57489c' THEN 'MEX - Insurgentes Sur'
   WHEN facility_id = '565273fe-0074-421d-b1e1-796d1e4390e1' THEN 'MEX - Napoles'
   WHEN facility_id = '864e0a3c-ecba-4305-b3ff-f22edd58c902' THEN 'MEX - Polanco'
   WHEN facility_id = 'bcb04949-3cda-4ca0-be39-b9f579e356ca' THEN 'MEX - Roma'
   WHEN facility_id = '9101d4f9-828f-45c5-b8b6-16986e76b374' THEN 'MEX - Pedregal'
   WHEN facility_id = 'b9545910-bca6-4b2a-a603-363721e7a23d' THEN 'MEX - Santa Fe'
   WHEN facility_id = 'da27f06f-01fd-4106-88d9-b3fd41446515' THEN 'MEX - Lindavista'
   
   WHEN facility_id = '394c0103-1c5f-424b-b398-68455fced5bb' THEN 'MTY - Cumbres'
   WHEN facility_id = '6d2d6c66-47aa-4f6e-ad75-58f68cc36c5f' THEN 'MTY - Leones NK'
   WHEN facility_id = '988c22e5-20c2-42a5-bc30-989c68ff5eee' THEN 'MTY - Tec'
   WHEN facility_id = '1b90b4ae-6d8f-4a73-bb9e-708c83387430' THEN 'MTY - Santa Maria'
   WHEN facility_id = '88774bba-2e34-4771-8e64-8f059adf02a3' THEN 'MTY - Valle Oriente'
   
   WHEN facility_id = 'a9a6c4be-32b7-47cf-b783-7a5227a89ed4' THEN 'GDL - Ciudad del Sol'
   WHEN facility_id = 'c66ce711-5486-433f-a9b7-1b36ae3a6dee' THEN 'GDL - Av. Mexico'
    
   WHEN facility_id = '7b5c774f-7739-4d8e-a392-d49547d45fc3' THEN 'BHZ - Contagem Industrial'
   WHEN facility_id = '554263c6-bde4-4d4c-92a7-21ed77e122cd' THEN 'BHZ - Paracatu'
   WHEN facility_id = '522c745a-fa86-42de-83ca-227cbd3cb353' THEN 'BHZ - Arthur Guimaraes' 
   WHEN facility_id = '4518d99c-fa47-4da7-b5f5-c071c78b5ac2' THEN 'BHZ - Tancredo Neves'
   
   WHEN facility_id = '81e6538d-c45b-4479-92e8-fc4e04373e12' THEN 'RIO - Jose Bonifacio'
   WHEN facility_id = '466abb82-57b9-4e0d-9c5b-d1ef6c7f92bb' THEN 'RIO - Barra' 
   WHEN facility_id = '0ee5dccf-ed5b-465c-9858-b53351353e3b' THEN 'RIO - Camerino'
   WHEN facility_id = '976ae838-18e0-40bb-a8dc-360353c01b6d' THEN 'RIO - Niteroi'
   WHEN facility_id = 'f7a65e9f-3342-48d2-a0fb-e9e4633c4bf6' THEN 'RIO - Tijuca'
   WHEN facility_id = '3759ad21-4537-48ea-b1f1-48809e8a03ac' then 'RIO - Botafogo'
   
   WHEN facility_id = '6c601837-403c-425f-a51c-31428ed1a67e' THEN 'SAO - Rua do Acre'
   WHEN facility_id = '87a40094-cf48-4032-8166-e4cee61bce2f' THEN 'SAO - Dino'
   WHEN facility_id = '4326199f-628a-4b00-a6e6-054fb5182c64' THEN 'SAO - Guararapes (Brooklin)'
   WHEN facility_id = 'dcc461ae-c80c-46cc-a5dc-4a5a72761510' THEN 'SAO - Lapa Clelia'
   WHEN facility_id = '7aae3115-311b-449f-88f4-0f1612ecfcee' THEN 'SAO - Borba'
   WHEN facility_id = '88aaaf99-a444-4e0f-99f6-267b635faa53' THEN 'SAO - Vargas'
   WHEN facility_id = '1840180b-9a3b-4fb6-b35c-ef4e6785b26e' THEN 'SAO - Abelardo'
   WHEN facility_id = 'a63559d1-93c6-4f60-b683-1e207ccda10a' THEN 'SAO - Batista'
  -- WHEN facility_id = '89cf86b9-9e74-4657-86df-2a851492af88' THEN 'SAO - Guarulhos'
   when facility_id = '14d9bb92-6376-4d69-adb6-64f97e4b3561' THEN 'SAO - Ferreira'
    WHEN facility_id = '89cf86b9-9e74-4657-86df-2a851492af88' THEN 'SAO - Espanhola'
    WHEN facility_id = '812e6913-d78e-4dbc-877d-16993853aa7e' THEN 'SAO - Domingos'
  
   WHEN facility_id = 'c559b06b-e193-495c-85cd-a5d1608ca27d' THEN 'BSB - Aguas Claras'
   
   
  
 
   WHEN facility_id = 'db3a588c-59ac-458e-84de-9e1c8ef5e3e8' THEN 'SCL - Recoleta'

  
END
) AS facility_sf_name,
facility_name,
organization_id,
brand_id,
co.order_id,
facility_id,
order_display_id,
ofo_slug, store_link_service_slug,
facility_country_code as country_code, 
is_cancelled,
oc_order_issue_count,
order_issue_details,
hoi_order_issue_details,
oc_order_issue_details,
order_issue_types,
hoi_order_issue_types,
oc_order_issue_types,

subtotal,
total,

commission, discount, tip, tip_for_restaurant, tip_for_courier, delivery_fee, marketing_fee, service_fee

,ROW_NUMBER() OVER (PARTITION BY ofo_slug, external_order_id, order_items_quantity, subtotal ORDER BY co.order_id) as row_number

,cast(local_accepted_at_last_ts as TIME ) as accepted_local_time,
local_processed_at_ts,local_ordered_at_ts,local_accepted_at_ts, local_in_handoff_at_ts,
local_handed_off_at_ts,local_printed_at_ts,local_trashed_at_ts,local_pickup_picked_up_at_ts,
local_courier_checked_in_at_ts,local_courier_dropped_off_at_ts,external_order_id,fulfillment_state,fulfillment_mode,
pickup_state,scheduling_type,ofo_status,order_status
,cancellation_state,cancellation_reason as facility_cancellation_reason
,ofo_cancellation_reason,order_rating, order_review_full_text,location_state,

case when location_state='LOCATION_HANDED_OFF' then 1
  else 0
  end
as is_handed_off_num,

Case When in_handoff_at IS NOT NULL THEN 1
  else 0
  END 
as is_in_handoff_num,




CASE
        WHEN is_cancelled THEN 'cancel'
        WHEN order_issue_types IS NOT NULL THEN 'issue'
        ELSE 'success'
        END AS defect_type,

CASE
        WHEN is_cancelled AND cancellation_reason = 'UNAVAILABLE_ITEM' THEN 'item_unavailable'
        WHEN is_cancelled AND confirmation_state <> 'CONFIRMATION_CONFIRMED' THEN 'confirmation_failed'
        WHEN is_cancelled AND  location_state!='LOCATION_HANDED_OFF' THEN 'handoff_failed'
        WHEN is_cancelled AND time_to_last_in_handoff_millis > 600000 THEN 'pick_over_10' --Pick time was over 10 minutes long
        WHEN is_cancelled THEN 'other_cancel'
        WHEN array_join(order_issue_types, ', ') LIKE '%WRONG_ORDER%' THEN 'order_wrong'
        WHEN array_join(order_issue_types, ', ') LIKE '%MISSING_ITEM%' OR array_join(order_issue_types, ', ') LIKE '%WRONG_QUANTITY%' THEN 'item_missing'
        WHEN array_join(order_issue_types, ', ') LIKE '%WRONG_ITEM%' OR array_join(order_issue_types, ', ') LIKE '%WRONG_SIZE%' THEN 'item_wrong'
        WHEN array_join(order_issue_types, ', ') LIKE '%INCOMPLETE%' THEN 'order_incomplete'
        ELSE 'other_issue'
      END AS defect_reason,

COALESCE(IF(adjustment IS NOT NULL AND (is_cancelled OR order_issue_types IS NOT NULL), adjustment < 0), FALSE) AS is_refunds
,time_to_last_in_handoff_millis BETWEEN 1 AND 600000 AS order_10picked --TRUE if picked in under 20 minutes

,order_source, hoi_only_order_type,order_items_quantity,scan_out.order_scanned_out,


printed_at_ts, in_handoff_at_ts, handed_off_at_ts, courier_checked_in_at_ts, courier_dropped_off_at_ts, courier_arrived_at_ts

,scanned_items

,customer_note

,concat('https://tools.cloudkitchens.com/order-inspector/order-details/',co.order_id,'/overview') as link


FROM hudi_ingest.analytics_views.customer_orders_with_hoi_curr co
JOIN hudi_ingest.scratch.dim_countries dc 
   ON co.facility_country_code = dc.country_code AND region = 'LATAM'
   
left join scan_out
on co.order_id=scan_out.order_id






--aqui se filtra la fecha    
WHERE co.day_partition >= DATE_FORMAT(DATE_TRUNC('month', current_date) - interval '12' month, '%Y-%m-%d')

AND reference_time/1000 IS NOT NULL 
AND DATE(from_unixtime(reference_time/1000, facility_timezone)) >= DATE '2021-01-15'
AND tenant_type IN ('VIRTUAL', 'CLOUD_RETAIL')

--AND is_cancelled

AND facility_id IN ( -- 
-- ANDEAN
    --PERU
        --LIMA
           -- 'd0c82479-ad1c-4a7e-aaa5-d83ee49e9df7', -- LIM - Surquillo
            --'6d2d117b-3fa9-4a2c-85b5-9b86a758f849', -- LIM - Magdalena
    --ECUADOR
        -- QUITO
            --'d65136fe-862a-4da5-81ef-580c28f8c61a', -- UIO - 10 de Agosto
-- CCAC
    -- COLOMBIA
        -- BOGOTA
            '567933f5-72a7-4b7a-b6b0-9e1ff87f5bcf', -- BOG - Prado
          --'199c5433-d863-4d5e-bb6e-a53d91e0605f', -- Modelia  BOGOTA
            '4d78d1e1-5bce-4307-8175-51c816120fce', -- BOG - Castellana
            'b2f57126-8e64-4b11-964f-c497a3b53571', -- BOG - Cocinas de la 73
            'a5852445-e223-46c9-be99-ba33f204bcd1', -- BOG - Puente Aranda
            '76a6ef30-ddb2-4178-8c37-86c0d10db4b3', -- BOG - Atabanza
            'dd16bf0e-8a89-4f21-bc02-1c9793227502', -- BOG - San Felipe
            '3a6e7382-f2c0-4ebc-8f12-61f4d44dee82', -- BOG - Cedritos
            '757182ff-7a54-4571-91d4-0027253c0117', --BOG salitre
            --medellin
            '0016d9fc-4f27-4a05-88cb-90cd1cdf311f',--de moda
        -- CALI    
            '46fb5e19-3a79-4eda-b119-dee3a331fd81', -- CLO - Chipichape
        -- BARRANQUILLA
            '58b818c6-615e-4316-b4a3-b9a6bb914c30', -- BAQ - Alto Prado
    -- COSTA RICA
        -- SAN JOSE
            --'9eede066-b7e7-43c4-907c-21ece1688b14', -- SJO - Guadalupe
           -- '967f0a2e-9c8e-4b35-929b-37420d9439c9', -- SJO - Pavas
-- MEXICO
        -- MEXICO CITY
            '47e2d054-8b82-420c-b1b0-18010f57489c', -- MEX - Insurgentes Sur
            '565273fe-0074-421d-b1e1-796d1e4390e1', -- MEX - Napoles
            '864e0a3c-ecba-4305-b3ff-f22edd58c902', -- MEX - Polanco
            'bcb04949-3cda-4ca0-be39-b9f579e356ca', -- MEX - Roma
            '9101d4f9-828f-45c5-b8b6-16986e76b374', -- MEX - Pedregal
            'b9545910-bca6-4b2a-a603-363721e7a23d', -- MEX - Santa Fe
            'da27f06f-01fd-4106-88d9-b3fd41446515', -- MEX - Lindavista
        -- MONTERREY
            '394c0103-1c5f-424b-b398-68455fced5bb', -- MTY - Cumbres
            '6d2d6c66-47aa-4f6e-ad75-58f68cc36c5f', -- MTY - Leones NK
            '988c22e5-20c2-42a5-bc30-989c68ff5eee', -- MTY - Tec
            '1b90b4ae-6d8f-4a73-bb9e-708c83387430', -- MTY - Santa Maria
            '88774bba-2e34-4771-8e64-8f059adf02a3', -- MTY - Valle Oriente
        -- GUADALAJARA
            'a9a6c4be-32b7-47cf-b783-7a5227a89ed4', -- GDL - Ciudad del Sol
            'c66ce711-5486-433f-a9b7-1b36ae3a6dee', -- GDL - av mexico
            
-- CHILE
        -- SANTIAGO
            --'db3a588c-59ac-458e-84de-9e1c8ef5e3e8', -- SCL - Recoleta

        
-- BRAZIL
        -- BELO HORIZONTE
            '7b5c774f-7739-4d8e-a392-d49547d45fc3', -- BHZ - Contagem Industrial
            '554263c6-bde4-4d4c-92a7-21ed77e122cd', -- BHZ - Paracatu
            '522c745a-fa86-42de-83ca-227cbd3cb353', -- BHZ - Arthur Guimaraes
            '4518d99c-fa47-4da7-b5f5-c071c78b5ac2', -- BHZ - Tancredo Neves
        -- RIO DE JANEIRO
            '81e6538d-c45b-4479-92e8-fc4e04373e12', -- RIO - Jose Bonifacio
            '466abb82-57b9-4e0d-9c5b-d1ef6c7f92bb', -- RIO - Barra
            '0ee5dccf-ed5b-465c-9858-b53351353e3b', -- RIO - Camerino
             '976ae838-18e0-40bb-a8dc-360353c01b6d', -- RIO - Niteroi
            'f7a65e9f-3342-48d2-a0fb-e9e4633c4bf6', -- RIO - Tijuca
            '3759ad21-4537-48ea-b1f1-48809e8a03ac', --rio botafogo
        -- SAO PAULO
            '6c601837-403c-425f-a51c-31428ed1a67e', -- SAO - Rua do Acre
            '87a40094-cf48-4032-8166-e4cee61bce2f', -- SAO - Dino
            '4326199f-628a-4b00-a6e6-054fb5182c64', -- SAO - Guararapes (Brooklin)
            'dcc461ae-c80c-46cc-a5dc-4a5a72761510', -- SAO - Lapa Clelia
            '7aae3115-311b-449f-88f4-0f1612ecfcee', -- SAO - Borba
            '1840180b-9a3b-4fb6-b35c-ef4e6785b26e', -- SAO - Abelardo
            '88aaaf99-a444-4e0f-99f6-267b635faa53',  -- SAO - Vargas
             'a63559d1-93c6-4f60-b683-1e207ccda10a', -- SAO - Batista
            --'89cf86b9-9e74-4657-86df-2a851492af88', -- SAO - Guarulhos
            '89cf86b9-9e74-4657-86df-2a851492af88', --sao espanhola
            '14d9bb92-6376-4d69-adb6-64f97e4b3561',-- SAO - Ferreira
            '812e6913-d78e-4dbc-877d-16993853aa7e', --sao domingos
            
                
        -- BRASILIA
            'c559b06b-e193-495c-85cd-a5d1608ca27d'  -- BSB - Aguas Claras
                
                )


)


Select created_month,country, ordenes.organization_id, ordenes.facility_id, ordenes.facility_sf_name

,CASE 
when org.deleted_at is not null then True

ELSE  False END

as org_deleted_skusku





, count (DISTINCT (order_id)) as total_orders
, count(DISTINCT CASE WHEN is_cancelled THEN order_id END) as cancelled_orders
,count (DISTINCT CASE WHEN defect_type = 'issue' THEN order_id END) as issue_orders
,count (DISTINCT CASE WHEN defect_type != 'success' THEN order_id END) as ODR_orders
,count(DISTINCT  CASE WHEN order_scanned_out THEN order_id END) as scanned_orders
,sum(subtotal) as total_gmv_local
,SUM(CASE WHEN is_cancelled THEN coalesce(subtotal, 0) ELSE 0 END) AS cancelled_gmv_local
,count (DISTINCT ofo_slug) as OFOs
,count(DISTINCT city) as cities
, count(DISTINCT facility_sf_name) as facilities
, AVG (order_items_quantity) as Avg_Items_per_Order
, AVG(order_rating) as avg_rating
, AVG(in_handoff_at_ts-printed_at_ts) as avg_picking_time
,  avg(date_diff('minute',printed_at_ts,in_handoff_at_ts) )as avg_picking_time_minutes 
,COUNT(DISTINCT CASE WHEN order_10picked = True THEN order_id END) as orders_10picked -- orders picked in under 10 minutes
from ordenes

left join hudi_ingest.processed.store_config_organization_view as org

on ordenes.organization_id=org.organization_id

WHERE  row_number=1

and ordenes.organization_id not in (

'45ad476b-9874-4513-8a29-1b3b04180fbd', --casa de pastor
'2f3aefc9-2f28-45df-8de5-2b52530abef1', --bento vk
'cb671780-a757-4db5-8782-100106cad446', --delichurros vk
'842a7a9f-e25c-433d-bb4f-c1912db1bb04', --guisamor
'8d09b777-805e-49cf-a172-27fbf2840dd1',--donas americanas
'46f35c4f-1bf1-479d-92c9-e729a40ba816',-- FREE-TANGAS & GAR-NACHAS - VK
'66279914-31e6-4df9-9c57-36153545b28a',-- iRobot


--tests:

'beb89975-2feb-4d0b-a61e-ac2a3b87bd18', --Org Test PiknPak
'24622819-678c-45fb-a5be-acece3bf9c73', -- CR Brazil - Teste Superstore
'b2d4fbf2-5ddc-46ad-8207-36b7bf77c88a', -- Conta Teste - Catherine
'fae923c9-8a1d-468e-9740-95c36c0fd4fc', --Conta Teste Pick n pack - Comercial
'804c6336-abc2-4293-81ee-0cea88c15029' -- warehouse and ops analyst



)


group by 1,2,3,4,5,6

"""
print("Querying data...")
opp_data = ds_trino.fetch_data(conn=conn, sql_str= opty_query, use_cache=False)
print("Data pulled!")
print(len(opp_data))

Querying data...
Data pulled!
6936


In [4]:
# Inspect 
opp_data.head(3)

,created_month,country,organization_id,facility_id,facility_sf_name,org_deleted_skusku,total_orders,cancelled_orders,issue_orders,ODR_orders,scanned_orders,total_gmv_local,cancelled_gmv_local,OFOs,cities,facilities,Avg_Items_per_Order,avg_rating,avg_picking_time,avg_picking_time_minutes,orders_10picked
0,2023-05-01,Brazil,e445cbbd-b698-43cf-9ccc-4362d07b4381,f7a65e9f-3342-48d2-a0fb-e9e4633c4bf6,RIO - Tijuca,False,435,13,0,13,424,17267.54,470.99,2,1,1,1.974713,4.803571,None,NaN,395
1,2022-09-01,Mexico,45474a0a-ca40-42ed-9ab2-1ad008a834b5,b9545910-bca6-4b2a-a603-363721e7a23d,MEX - Santa Fe,True,211,6,1,7,204,49277.76,401.55,2,1,1,2.203791,4.466667,0 00:02:52.720,2.508475,194
2,2022-10-01,Mexico,45474a0a-ca40-42ed-9ab2-1ad008a834b5,b9545910-bca6-4b2a-a603-363721e7a23d,MEX - Santa Fe,True,211,3,0,3,207,47727.46,330.00,2,1,1,2.483412,4.291667,0 00:03:40.696,3.130435,201


In [5]:
opp_data.total_gmv_local.sum()

2870159016.887624

Import customer names

In [6]:
client_query = """

SELECT organization_id,name 
FROM hudi_ingest.processed.store_config_organization_view

"""
print("Querying data...")
client_data = ds_trino.fetch_data(conn=conn, sql_str= client_query, use_cache=False)
print("Data pulled!")
print(len(client_data))

Querying data...
Data pulled!
339993


In [7]:
# Merge to opp data
opp_data = opp_data.merge(client_data, how = "left", on = "organization_id")

In [8]:
print(len(opp_data))

6936


In [9]:
opp_data.head()

,created_month,country,organization_id,facility_id,facility_sf_name,org_deleted_skusku,total_orders,cancelled_orders,issue_orders,ODR_orders,scanned_orders,total_gmv_local,cancelled_gmv_local,OFOs,cities,facilities,Avg_Items_per_Order,avg_rating,avg_picking_time,avg_picking_time_minutes,orders_10picked,name
0,2023-05-01,Brazil,e445cbbd-b698-43cf-9ccc-4362d07b4381,f7a65e9f-3342-48d2-a0fb-e9e4633c4bf6,RIO - Tijuca,False,435,13,0,13,424,17267.54,470.99,2,1,1,1.974713,4.803571,None,NaN,395,Brownice - Tijuca - Doces e Sobremesas - Pik n...
1,2022-09-01,Mexico,45474a0a-ca40-42ed-9ab2-1ad008a834b5,b9545910-bca6-4b2a-a603-363721e7a23d,MEX - Santa Fe,True,211,6,1,7,204,49277.76,401.55,2,1,1,2.203791,4.466667,0 00:02:52.720,2.508475,194,Stuffed
2,2022-10-01,Mexico,45474a0a-ca40-42ed-9ab2-1ad008a834b5,b9545910-bca6-4b2a-a603-363721e7a23d,MEX - Santa Fe,True,211,3,0,3,207,47727.46,330.00,2,1,1,2.483412,4.291667,0 00:03:40.696,3.130435,201,Stuffed
3,2022-11-01,Mexico,45474a0a-ca40-42ed-9ab2-1ad008a834b5,864e0a3c-ecba-4305-b3ff-f22edd58c902,MEX - Polanco,True,178,2,2,4,174,39460.89,250.00,2,1,1,2.494382,4.666667,None,NaN,150,Stuffed
4,2023-07-01,Brazil,ac0f7e69-d5fa-49c9-8c70-b3fb7523de43,4518d99c-fa47-4da7-b5f5-c071c78b5ac2,BHZ - Tancredo Neves,False,190,2,0,2,182,7508.76,139.87,1,1,1,2.178947,4.631579,None,NaN,154,Carolline Doceria - Pik n' Pak


Calculate metrics per customer per month

In [10]:
import pytz
import datetime
from datetime import timedelta

In [11]:
# Convert to time data the corrsponding columns

# Define a function to strip zeroes
def strip_zero(x):
    try:
        y=x.strip()
        return y[3:]
    except:
        return x

# Strip zeroes from avg_picking_time
opp_data["avg_picking_time"]= opp_data["avg_picking_time"].apply(strip_zero)

# Convert to datetime
opp_data['avg_picking_mins']=pd.to_datetime(opp_data['avg_picking_time'],errors='coerce').dt.minute+pd.to_datetime(opp_data['avg_picking_time'],errors='coerce').dt.second/60

In [12]:
# Create last updated column
opp_data["data_last_updated"] = datetime.datetime.now(pytz.timezone('Mexico/General'))

In [13]:
opp_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6936 entries, 0 to 6935
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype                         
---  ------                    --------------  -----                         
 0   created_month             6936 non-null   object                        
 1   country                   6936 non-null   object                        
 2   organization_id           6936 non-null   object                        
 3   facility_id               6936 non-null   object                        
 4   facility_sf_name          6936 non-null   object                        
 5   org_deleted_skusku        6936 non-null   bool                          
 6   total_orders              6936 non-null   int64                         
 7   cancelled_orders          6936 non-null   int64                         
 8   issue_orders              6936 non-null   int64                         
 9   ODR_orders                6936

In [14]:
# Convert string columns to numeric
opp_data["total_orders"] = pd.to_numeric(opp_data["total_orders"])
opp_data["cancelled_orders"] = pd.to_numeric(opp_data["cancelled_orders"])
opp_data["issue_orders"] = pd.to_numeric(opp_data["issue_orders"])
opp_data["ODR_orders"] = pd.to_numeric(opp_data["ODR_orders"])
opp_data["scanned_orders"] = pd.to_numeric(opp_data["scanned_orders"])

In [15]:
# Calculate ODR metrics
opp_data["pct_cancelled"] = opp_data["cancelled_orders"] / opp_data["total_orders"]
opp_data["pct_issues"] = opp_data["issue_orders"] / opp_data["total_orders"]
opp_data["pct_odr"] = opp_data["ODR_orders"] /  opp_data["total_orders"]
opp_data["pct_scanned"] = opp_data["scanned_orders"] / opp_data["total_orders"]

In [16]:
opp_data.head()

,created_month,country,organization_id,facility_id,facility_sf_name,org_deleted_skusku,total_orders,cancelled_orders,issue_orders,ODR_orders,scanned_orders,total_gmv_local,cancelled_gmv_local,OFOs,cities,facilities,Avg_Items_per_Order,avg_rating,avg_picking_time,avg_picking_time_minutes,orders_10picked,name,avg_picking_mins,data_last_updated,pct_cancelled,pct_issues,pct_odr,pct_scanned
0,2023-05-01,Brazil,e445cbbd-b698-43cf-9ccc-4362d07b4381,f7a65e9f-3342-48d2-a0fb-e9e4633c4bf6,RIO - Tijuca,False,435,13,0,13,424,17267.54,470.99,2,1,1,1.974713,4.803571,None,NaN,395,Brownice - Tijuca - Doces e Sobremesas - Pik n...,NaN,2023-08-22 14:59:32.775832-05:00,0.029885,0.000000,0.029885,0.974713
1,2022-09-01,Mexico,45474a0a-ca40-42ed-9ab2-1ad008a834b5,b9545910-bca6-4b2a-a603-363721e7a23d,MEX - Santa Fe,True,211,6,1,7,204,49277.76,401.55,2,1,1,2.203791,4.466667,0:02:52.720,2.508475,194,Stuffed,2.866667,2023-08-22 14:59:32.775832-05:00,0.028436,0.004739,0.033175,0.966825
2,2022-10-01,Mexico,45474a0a-ca40-42ed-9ab2-1ad008a834b5,b9545910-bca6-4b2a-a603-363721e7a23d,MEX - Santa Fe,True,211,3,0,3,207,47727.46,330.00,2,1,1,2.483412,4.291667,0:03:40.696,3.130435,201,Stuffed,3.666667,2023-08-22 14:59:32.775832-05:00,0.014218,0.000000,0.014218,0.981043
3,2022-11-01,Mexico,45474a0a-ca40-42ed-9ab2-1ad008a834b5,864e0a3c-ecba-4305-b3ff-f22edd58c902,MEX - Polanco,True,178,2,2,4,174,39460.89,250.00,2,1,1,2.494382,4.666667,None,NaN,150,Stuffed,NaN,2023-08-22 14:59:32.775832-05:00,0.011236,0.011236,0.022472,0.977528
4,2023-07-01,Brazil,ac0f7e69-d5fa-49c9-8c70-b3fb7523de43,4518d99c-fa47-4da7-b5f5-c071c78b5ac2,BHZ - Tancredo Neves,False,190,2,0,2,182,7508.76,139.87,1,1,1,2.178947,4.631579,None,NaN,154,Carolline Doceria - Pik n' Pak,NaN,2023-08-22 14:59:32.775832-05:00,0.010526,0.000000,0.010526,0.957895


In [17]:
# Inspect 
opp_data[opp_data["name"] == "Stuffed"].head(3)

,created_month,country,organization_id,facility_id,facility_sf_name,org_deleted_skusku,total_orders,cancelled_orders,issue_orders,ODR_orders,scanned_orders,total_gmv_local,cancelled_gmv_local,OFOs,cities,facilities,Avg_Items_per_Order,avg_rating,avg_picking_time,avg_picking_time_minutes,orders_10picked,name,avg_picking_mins,data_last_updated,pct_cancelled,pct_issues,pct_odr,pct_scanned
1,2022-09-01,Mexico,45474a0a-ca40-42ed-9ab2-1ad008a834b5,b9545910-bca6-4b2a-a603-363721e7a23d,MEX - Santa Fe,True,211,6,1,7,204,49277.76,401.55,2,1,1,2.203791,4.466667,0:02:52.720,2.508475,194,Stuffed,2.866667,2023-08-22 14:59:32.775832-05:00,0.028436,0.004739,0.033175,0.966825
2,2022-10-01,Mexico,45474a0a-ca40-42ed-9ab2-1ad008a834b5,b9545910-bca6-4b2a-a603-363721e7a23d,MEX - Santa Fe,True,211,3,0,3,207,47727.46,330.00,2,1,1,2.483412,4.291667,0:03:40.696,3.130435,201,Stuffed,3.666667,2023-08-22 14:59:32.775832-05:00,0.014218,0.000000,0.014218,0.981043
3,2022-11-01,Mexico,45474a0a-ca40-42ed-9ab2-1ad008a834b5,864e0a3c-ecba-4305-b3ff-f22edd58c902,MEX - Polanco,True,178,2,2,4,174,39460.89,250.00,2,1,1,2.494382,4.666667,None,NaN,150,Stuffed,NaN,2023-08-22 14:59:32.775832-05:00,0.011236,0.011236,0.022472,0.977528


**Pull Salesforce Data for MRR**

In [18]:
# Install Simplesalesforce
# !pip install simple-salesforce 
from simple_salesforce import Salesforce, SalesforceLogin, SFType
import json

In [19]:
# Import credentials
path = "/home/shared/joseph.jensen/Alan Data Pipelines/Credentials/sf_auth_JJ.json"

loginInfo = json.load(open(path))
session_id, instance = SalesforceLogin(username= loginInfo["username"],
                                                 password= loginInfo["password"],
                                                 security_token= loginInfo["security_token"],
                                                 domain= "login")

# Set SF connection
sf = Salesforce(instance= instance, session_id= session_id)

In [20]:
# Query category data from Salesforce
sf_query = """
SELECT 
Account.Org_Id_Otter__c,
Account_Name_Jira__c, 
account.Current_Customer__c, 
Opportunity_ID_18__c, 
Name, Facility__r.Name,  
Cuisine_Type__c,  
Id, 
Kitchen_City__c, 
Kitchen_Country__c, 
Is_Churned__c,
Churn_Date__c,
Processing_Fee__c, 
Monthly_Shelf_Fee_Dry_Storage__c,
Monthly_Shelf_Fee_Frozen_Storage__c,
Monthly_Shelf_Fee_Cold_Storage__c,
Closed_Won_Date__c,
Move_In_Date__c, 
Start_Date__c,
Loss_Reason__c, 
Specific_Lost_Reason__c, 
Lost_Explanation__c,
CR_Opp_Opted_out_Affiliate__c,
CR_Opp_Restrictions_Affiliate__c,
CR_Opp_Type_of_Processing_Fee__c,
account.CR_Account_Subcategory__c, 
account.CR_Acc_Social_Media_Presence_Numeric__c, 
account.CR_Type_of_Customer__c,
Storage_Amount_Dry_CU_FT__c, 
Storage_Amount_Frozen_CU_FT__c, 
Storage_Amount_Cold_CU_FT__c


FROM
Opportunity  

WHERE
RecordType.Name in ('Virtual', 'CloudRetail','virtual;cloudretail') AND (
Opp_Owner_Region__c in ('LATAM') OR Kitchen_Country__c IN ('Mexico', 'Colombia', 'Brazil'))
AND StageName = 'Closed Won' ORDER BY Closed_Won_Date__c DESC

"""


print("Querying data...")

# Query users to dataframe
sf_data = pd.DataFrame(sf.query_all(sf_query)['records'])
print("Data pulled!")

# Inspect 
sf_data.head()
print("Rows:", len(sf_data))

Querying data...
Data pulled!
Rows: 2775


In [21]:
# Inspect
sf_data.head()
print(len(sf_data))

2775


In [22]:
# Extract account details from accounts list in accounts column
account_name = [] # lista orgs id
for i in sf_data['Facility__r']:
    try:
        j=(i.get('Name'))
        #print(j)
        account_name.append(j)
    except:
        account_name.append(None)

# Append extracted lists to DF
sf_data['Facility__r']= account_name

In [23]:
# Rename new facility name field to match old name
sf_data.rename(columns = {"Facility__r":"Facility_Name__c"}, inplace = True)

In [24]:
sf_data.head()
print(len(sf_data))

2775


Query data from account table in SF

In [25]:
acct_query = """
SELECT
Account.LAM_TierScrape__c, 
Account.CSM__c,
Org_Id_Otter__c,
CR_Account_Categories__c,
CR_Account_Sub_category__c,
CR_Account_Cuisine_Type__c,
CR_Type_of_Customer__c,
CR_Acc_Social_Media_Presence_Numeric__c,
id

FROM
Account
"""


print("Querying data...")

# Query users to dataframe
acct_query = pd.DataFrame(sf.query_all(acct_query)['records'])
print("Data pulled!")

# Inspect 
acct_query.head()
print("Rows:", len(acct_query))

Querying data...
Data pulled!
Rows: 63797


In [26]:
acct_query.head(2)

,attributes,LAM_TierScrape__c,CSM__c,Org_Id_Otter__c,CR_Account_Categories__c,CR_Account_Sub_category__c,CR_Account_Cuisine_Type__c,CR_Type_of_Customer__c,CR_Acc_Social_Media_Presence_Numeric__c,Id
0,"{'type': 'Account', 'url': '/services/data/v52...",None,None,None,None,None,None,None,NaN,0015G00001VbfshQAB
1,"{'type': 'Account', 'url': '/services/data/v52...",None,None,None,None,None,None,None,NaN,0015G00001Vbgz1QAB


In [27]:
# Create bucket for social media presence with same SF field name
social_media = []

for s in acct_query["CR_Acc_Social_Media_Presence_Numeric__c"]:
    if s >= 100000:
        s = '>100k'
    elif s >= 50000:
        s = '50k-100k'
    elif s >= 50000:
        s = '20k-50k'
    elif s >= 10000:
        s = '10k-20k'
    elif s >= 7000:
        s = '7k-10k'
    elif s >= 5000:
        s = '5k-7k'
    elif s >= 3000:
        s = '3k-5k'
    elif s >= 1000:
        s = '1k-3k'
    else:
        s = '<1k'
    social_media.append(s)

# Create new col with ex SF name
acct_query["CR_Acc_Social_Media_Presence__c"] = social_media

In [28]:
acct_query = acct_query[["Org_Id_Otter__c", "CSM__c", "CR_Account_Categories__c","CR_Account_Sub_category__c", "CR_Account_Cuisine_Type__c", "CR_Acc_Social_Media_Presence__c", "LAM_TierScrape__c","CR_Type_of_Customer__c","CR_Acc_Social_Media_Presence_Numeric__c"]]
acct_query.head()

,Org_Id_Otter__c,CSM__c,CR_Account_Categories__c,CR_Account_Sub_category__c,CR_Account_Cuisine_Type__c,CR_Acc_Social_Media_Presence__c,LAM_TierScrape__c,CR_Type_of_Customer__c,CR_Acc_Social_Media_Presence_Numeric__c
0,None,None,None,None,None,<1k,None,None,NaN
1,None,None,None,None,None,<1k,None,None,NaN
2,ab4eb478-b0bb-4fd5-8de9-8c3274b76911,None,CR - Frozen Food,CR - Meat/Shrimp/Chicken,CR - Baked Goods,>100k,Tier 3,None,114000.0
3,None,None,None,None,None,<1k,None,None,NaN
4,None,None,None,None,None,<1k,None,None,NaN


In [29]:
acct_query = acct_query.drop_duplicates(subset = "Org_Id_Otter__c")

In [30]:
print(len(acct_query))

1781


In [31]:
# Extract account details from accounts list in accounts column
org_ids = [] # lista orgs id
actual_cust =[] #lista cliente actual
has_churned = [] # lista has churned
for i in sf_data['Account']:
    try:
        j=(i.get('Org_Id_Otter__c'))
        #print(j)
        org_ids.append(j)
        k=(i.get('Current_Customer__c'))
        actual_cust.append(k)
    except:
        org_ids.append(None)
        actual_cust.append(None)

In [32]:
# Append extracted lists to DF
sf_data['sfdc_org_id']= org_ids
sf_data['current_customer']= actual_cust

In [33]:
# Fill NAs with bfill 
sf_data['org_id']= sf_data[["sfdc_org_id"]].bfill(axis=1).iloc[:, 0]

In [34]:
# Inspect
print("Rows:", len(sf_data))
sf_data.head()

Rows: 2775


,attributes,Account,Account_Name_Jira__c,Opportunity_ID_18__c,Name,Facility_Name__c,Cuisine_Type__c,Id,Kitchen_City__c,Kitchen_Country__c,Is_Churned__c,Churn_Date__c,Processing_Fee__c,Monthly_Shelf_Fee_Dry_Storage__c,Monthly_Shelf_Fee_Frozen_Storage__c,Monthly_Shelf_Fee_Cold_Storage__c,Closed_Won_Date__c,Move_In_Date__c,Start_Date__c,Loss_Reason__c,Specific_Lost_Reason__c,Lost_Explanation__c,CR_Opp_Opted_out_Affiliate__c,CR_Opp_Restrictions_Affiliate__c,CR_Opp_Type_of_Processing_Fee__c,Storage_Amount_Dry_CU_FT__c,Storage_Amount_Frozen_CU_FT__c,Storage_Amount_Cold_CU_FT__c,sfdc_org_id,current_customer,org_id
0,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",TEST DANIELA,0066T00001BfHs7QAF,TEST DANIELA - TP - Trial,LA - Hollywood (Western),None,0066T00001BfHs7QAF,Los Angeles,United States,False,None,NaN,NaN,NaN,NaN,None,None,2023-06-19,None,None,None,True,False,None,NaN,NaN,NaN,b2d4fbf2-5ddc-46ad-8207-36b7bf77c88a,False,b2d4fbf2-5ddc-46ad-8207-36b7bf77c88a
1,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",Nieve De Olla,0066T00001C0a16QAB,Nieve De Olla - MEX - Roma,MEX - Roma,None,0066T00001C0a16QAB,Mexico City,Mexico,False,None,10.0,0.0,1250.0,0.0,None,None,2023-07-01,None,None,None,True,False,Percentage / Order,0.0,4.0,0.0,a01f793e-ab3c-4de8-b99e-500be6d2977e,False,a01f793e-ab3c-4de8-b99e-500be6d2977e
2,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",La Licoreria Distribuidora,0066T00001D6dObQAJ,La Licoreria Distribuidora - MEX - Roma,MEX - Roma,None,0066T00001D6dObQAJ,Mexico City,Mexico,False,None,7.0,6000.0,0.0,0.0,None,None,None,None,None,None,True,False,Fixed Fee / Order,48.0,0.0,0.0,7d2329f1-fd48-49b5-8b1f-0400316f1ffd,False,7d2329f1-fd48-49b5-8b1f-0400316f1ffd
3,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",The Healthy Bakery,0066T00001C0Zo2QAF,The Healthy Bakery - MEX - Insurgentes Sur,MEX - Insurgentes Sur,None,0066T00001C0Zo2QAF,Mexico City,Mexico,False,None,9.0,0.0,2000.0,0.0,None,None,2023-07-01,None,None,None,True,False,Percentage / Order,0.0,4.0,0.0,53f797b7-d31f-493f-a41f-c32ec09a0b92,False,53f797b7-d31f-493f-a41f-c32ec09a0b92
4,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",CUPCAKESLAND,0066T00001C0ZxiQAF,CUPCAKESLAND - MEX - Insurgentes Sur,MEX - Insurgentes Sur,None,0066T00001C0ZxiQAF,Mexico City,Mexico,False,None,8.0,0.0,0.0,1000.0,None,None,2023-07-01,None,None,None,True,False,Percentage / Order,0.0,0.0,4.0,c17f5cf9-88a9-445e-b777-58a88bf9efca,False,c17f5cf9-88a9-445e-b777-58a88bf9efca


In [35]:
# Merge data
sf_data = sf_data.merge(acct_query, how = "left", left_on = ["org_id"], right_on = ["Org_Id_Otter__c"])

In [36]:
print(len(sf_data))

2775


In [37]:
sf_data.head()

,attributes,Account,Account_Name_Jira__c,Opportunity_ID_18__c,Name,Facility_Name__c,Cuisine_Type__c,Id,Kitchen_City__c,Kitchen_Country__c,Is_Churned__c,Churn_Date__c,Processing_Fee__c,Monthly_Shelf_Fee_Dry_Storage__c,Monthly_Shelf_Fee_Frozen_Storage__c,Monthly_Shelf_Fee_Cold_Storage__c,Closed_Won_Date__c,Move_In_Date__c,Start_Date__c,Loss_Reason__c,Specific_Lost_Reason__c,Lost_Explanation__c,CR_Opp_Opted_out_Affiliate__c,CR_Opp_Restrictions_Affiliate__c,CR_Opp_Type_of_Processing_Fee__c,Storage_Amount_Dry_CU_FT__c,Storage_Amount_Frozen_CU_FT__c,Storage_Amount_Cold_CU_FT__c,sfdc_org_id,current_customer,org_id,Org_Id_Otter__c,CSM__c,CR_Account_Categories__c,CR_Account_Sub_category__c,CR_Account_Cuisine_Type__c,CR_Acc_Social_Media_Presence__c,LAM_TierScrape__c,CR_Type_of_Customer__c,CR_Acc_Social_Media_Presence_Numeric__c
0,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",TEST DANIELA,0066T00001BfHs7QAF,TEST DANIELA - TP - Trial,LA - Hollywood (Western),None,0066T00001BfHs7QAF,Los Angeles,United States,False,None,NaN,NaN,NaN,NaN,None,None,2023-06-19,None,None,None,True,False,None,NaN,NaN,NaN,b2d4fbf2-5ddc-46ad-8207-36b7bf77c88a,False,b2d4fbf2-5ddc-46ad-8207-36b7bf77c88a,b2d4fbf2-5ddc-46ad-8207-36b7bf77c88a,None,CR - Not Listed,None,CR - Not Listed,<1k,Tier 3,None,NaN
1,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",Nieve De Olla,0066T00001C0a16QAB,Nieve De Olla - MEX - Roma,MEX - Roma,None,0066T00001C0a16QAB,Mexico City,Mexico,False,None,10.0,0.0,1250.0,0.0,None,None,2023-07-01,None,None,None,True,False,Percentage / Order,0.0,4.0,0.0,a01f793e-ab3c-4de8-b99e-500be6d2977e,False,a01f793e-ab3c-4de8-b99e-500be6d2977e,a01f793e-ab3c-4de8-b99e-500be6d2977e,0055G0000080B5LQAU,CR - Frozen Desserts,CR - Shaved Ice,"CR - Ice Cream, Popsicle, Cookie Dough & Gelato",10k-20k,Tier 3,Local,24100.0
2,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",La Licoreria Distribuidora,0066T00001D6dObQAJ,La Licoreria Distribuidora - MEX - Roma,MEX - Roma,None,0066T00001D6dObQAJ,Mexico City,Mexico,False,None,7.0,6000.0,0.0,0.0,None,None,None,None,None,None,True,False,Fixed Fee / Order,48.0,0.0,0.0,7d2329f1-fd48-49b5-8b1f-0400316f1ffd,False,7d2329f1-fd48-49b5-8b1f-0400316f1ffd,7d2329f1-fd48-49b5-8b1f-0400316f1ffd,0055G000006QIsHQAW,CR - Alcoholic Beverages,CR - Spirits,None,<1k,Tier 2,Local,0.0
3,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",The Healthy Bakery,0066T00001C0Zo2QAF,The Healthy Bakery - MEX - Insurgentes Sur,MEX - Insurgentes Sur,None,0066T00001C0Zo2QAF,Mexico City,Mexico,False,None,9.0,0.0,2000.0,0.0,None,None,2023-07-01,None,None,None,True,False,Percentage / Order,0.0,4.0,0.0,53f797b7-d31f-493f-a41f-c32ec09a0b92,False,53f797b7-d31f-493f-a41f-c32ec09a0b92,53f797b7-d31f-493f-a41f-c32ec09a0b92,0055G0000080B5LQAU,CR - Baked Goods,CR - Cupcakes,CR - Baked Goods,10k-20k,Tier 2,Mid-Market,13800.0
4,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",CUPCAKESLAND,0066T00001C0ZxiQAF,CUPCAKESLAND - MEX - Insurgentes Sur,MEX - Insurgentes Sur,None,0066T00001C0ZxiQAF,Mexico City,Mexico,False,None,8.0,0.0,0.0,1000.0,None,None,2023-07-01,None,None,None,True,False,Percentage / Order,0.0,0.0,4.0,c17f5cf9-88a9-445e-b777-58a88bf9efca,False,c17f5cf9-88a9-445e-b777-58a88bf9efca,c17f5cf9-88a9-445e-b777-58a88bf9efca,0055G0000080B5LQAU,CR - Baked Goods,CR - Cupcakes,CR - Baked Goods,1k-3k,Tier 3,None,1663.0


**Calculating MRR**

- local currency
- merging fx rates and calculating mrr in usd

**Merging SF data to Superset Data**

In [38]:
# Order by closed_won date desc.
sf_data["Closed_Won_Date__c"] = pd.to_datetime(sf_data["Closed_Won_Date__c"])
sf_data = sf_data.sort_values(by = "Closed_Won_Date__c", ascending = False).reset_index(drop = True)

# Inspect 
sf_data.head()

,attributes,Account,Account_Name_Jira__c,Opportunity_ID_18__c,Name,Facility_Name__c,Cuisine_Type__c,Id,Kitchen_City__c,Kitchen_Country__c,Is_Churned__c,Churn_Date__c,Processing_Fee__c,Monthly_Shelf_Fee_Dry_Storage__c,Monthly_Shelf_Fee_Frozen_Storage__c,Monthly_Shelf_Fee_Cold_Storage__c,Closed_Won_Date__c,Move_In_Date__c,Start_Date__c,Loss_Reason__c,Specific_Lost_Reason__c,Lost_Explanation__c,CR_Opp_Opted_out_Affiliate__c,CR_Opp_Restrictions_Affiliate__c,CR_Opp_Type_of_Processing_Fee__c,Storage_Amount_Dry_CU_FT__c,Storage_Amount_Frozen_CU_FT__c,Storage_Amount_Cold_CU_FT__c,sfdc_org_id,current_customer,org_id,Org_Id_Otter__c,CSM__c,CR_Account_Categories__c,CR_Account_Sub_category__c,CR_Account_Cuisine_Type__c,CR_Acc_Social_Media_Presence__c,LAM_TierScrape__c,CR_Type_of_Customer__c,CR_Acc_Social_Media_Presence_Numeric__c
0,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",Shoppifit,0066T00001C0st4QAB,Shoppifit - MED - De Moda,MED - DModa,None,0066T00001C0st4QAB,Medellin,Colombia,False,None,0.0,0.0,0.0,0.0,2023-08-18,None,2023-09-18,None,None,None,True,False,Percentage / Order,8.0,0.0,0.0,None,False,None,None,None,None,None,None,<1k,None,None,NaN
1,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",Shoppifit,0066T00001D7kTaQAJ,Shoppifit - BOG - San Felipe,BOG - San Felipe,None,0066T00001D7kTaQAJ,Bogota,Colombia,False,None,0.0,0.0,0.0,0.0,2023-08-18,None,2023-09-18,None,None,None,True,False,Percentage / Order,8.0,0.0,0.0,None,False,None,None,None,None,None,None,<1k,None,None,NaN
2,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",Food to Save,0066T00001D7txlQAB,Food to Save - BHZ - Paracatu,BHZ - Paracatu,None,0066T00001D7txlQAB,Sao Paulo,Brazil,False,None,0.0,0.0,0.0,0.0,2023-08-18,2023-07-26,2023-08-24,None,None,None,True,False,None,48.0,0.0,0.0,f9373a87-ab71-4198-bc2c-93fa2a27bcf8,False,f9373a87-ab71-4198-bc2c-93fa2a27bcf8,f9373a87-ab71-4198-bc2c-93fa2a27bcf8,0055G0000080AXnQAM,CR - Packaged Food / Ready to Eat,CR - Marmitas,CR - Packaged Food / Ready to Eat,>100k,Tier 1,Local,135000.0
3,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",Shoppifit,0066T00001D7kShQAJ,Shoppifit - BOG - Atabanza,BOG - Atabanza,None,0066T00001D7kShQAJ,Bogota,Colombia,False,None,0.0,0.0,0.0,0.0,2023-08-18,None,2023-09-18,None,None,None,True,False,Percentage / Order,8.0,0.0,0.0,None,False,None,None,None,None,None,None,<1k,None,None,NaN
4,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",Santa Rita ( helados y paletas ) - Empresa Activa,0066T00001D7ZWJQA3,Santa Rita ( helados y paletas ) - Empresa Act...,MEX - Polanco,None,0066T00001D7ZWJQA3,Mexico City,Mexico,False,None,8.0,0.0,1500.0,0.0,2023-08-17,None,2023-09-15,None,None,None,True,False,None,0.0,4.0,0.0,b368a673-374e-4990-b456-06fecf6ad9c9,False,b368a673-374e-4990-b456-06fecf6ad9c9,b368a673-374e-4990-b456-06fecf6ad9c9,None,CR - Frozen Desserts,CR - Ice Cream,None,10k-20k,None,Local,12000.0


In [39]:
# Dropping duplicates
print("Rows:", len(sf_data))
sf_data = sf_data.drop_duplicates(subset= ["org_id", "Facility_Name__c"], keep = "first").reset_index(drop=True)

Rows: 2775


In [40]:
# Merging SF to SS data
combined_data = opp_data.merge(sf_data,how='left', left_on=['organization_id',"facility_sf_name"], right_on=['org_id', "Facility_Name__c"])

In [41]:
# Inspect length
print("Rows:", len(opp_data))

Rows: 6936


In [42]:
# Creating MRR local column
combined_data["MRR_local"] = combined_data["Monthly_Shelf_Fee_Dry_Storage__c"] + combined_data["Monthly_Shelf_Fee_Frozen_Storage__c"] + combined_data["Monthly_Shelf_Fee_Cold_Storage__c"]

In [43]:
# Inspect length
print("Combined data rows:", len(combined_data))

Combined data rows: 6936


In [44]:
combined_data.head()

,created_month,country,organization_id,facility_id,facility_sf_name,org_deleted_skusku,total_orders,cancelled_orders,issue_orders,ODR_orders,scanned_orders,total_gmv_local,cancelled_gmv_local,OFOs,cities,facilities,Avg_Items_per_Order,avg_rating,avg_picking_time,avg_picking_time_minutes,orders_10picked,name,avg_picking_mins,data_last_updated,pct_cancelled,pct_issues,pct_odr,pct_scanned,attributes,Account,Account_Name_Jira__c,Opportunity_ID_18__c,Name,Facility_Name__c,Cuisine_Type__c,Id,Kitchen_City__c,Kitchen_Country__c,Is_Churned__c,Churn_Date__c,Processing_Fee__c,Monthly_Shelf_Fee_Dry_Storage__c,Monthly_Shelf_Fee_Frozen_Storage__c,Monthly_Shelf_Fee_Cold_Storage__c,Closed_Won_Date__c,Move_In_Date__c,Start_Date__c,Loss_Reason__c,Specific_Lost_Reason__c,Lost_Explanation__c,CR_Opp_Opted_out_Affiliate__c,CR_Opp_Restrictions_Affiliate__c,CR_Opp_Type_of_Processing_Fee__c,Storage_Amount_Dry_CU_FT__c,Storage_Amount_Frozen_CU_FT__c,Storage_Amount_Cold_CU_FT__c,sfdc_org_id,current_customer,org_id,Org_Id_Otter__c,CSM__c,CR_Account_Categories__c,CR_Account_Sub_category__c,CR_Account_Cuisine_Type__c,CR_Acc_Social_Media_Presence__c,LAM_TierScrape__c,CR_Type_of_Customer__c,CR_Acc_Social_Media_Presence_Numeric__c,MRR_local
0,2023-05-01,Brazil,e445cbbd-b698-43cf-9ccc-4362d07b4381,f7a65e9f-3342-48d2-a0fb-e9e4633c4bf6,RIO - Tijuca,False,435,13,0,13,424,17267.54,470.99,2,1,1,1.974713,4.803571,None,NaN,395,Brownice - Tijuca - Doces e Sobremesas - Pik n...,NaN,2023-08-22 14:59:32.775832-05:00,0.029885,0.000000,0.029885,0.974713,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",Brownice,0065G00000j7FmKQAU,Brownice - RIO - Tijuca,RIO - Tijuca,CR - Baked Goods,0065G00000j7FmKQAU,Rio de Janeiro,Brazil,False,None,5.0,930.0,0.0,1395.0,2021-12-28,2022-05-02,2022-05-02,None,None,None,False,False,None,8.0,0.0,12.0,e445cbbd-b698-43cf-9ccc-4362d07b4381,False,e445cbbd-b698-43cf-9ccc-4362d07b4381,e445cbbd-b698-43cf-9ccc-4362d07b4381,0056T000009cfwEQAQ,CR - Baked Goods,CR - Brownies,CR - Baked Goods,10k-20k,Tier 2,None,12000.0,2325.0
1,2022-09-01,Mexico,45474a0a-ca40-42ed-9ab2-1ad008a834b5,b9545910-bca6-4b2a-a603-363721e7a23d,MEX - Santa Fe,True,211,6,1,7,204,49277.76,401.55,2,1,1,2.203791,4.466667,0:02:52.720,2.508475,194,Stuffed,2.866667,2023-08-22 14:59:32.775832-05:00,0.028436,0.004739,0.033175,0.966825,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",Stuffed,0065G00000nahZ7QAI,Stuffed - MEX - Santa Fe,MEX - Santa Fe,CR - Baked Goods,0065G00000nahZ7QAI,Mexico City,Mexico,True,2022-12-31,10.0,1500.0,0.0,1000.0,2022-02-24,2022-03-24,2022-03-24,None,None,None,False,False,None,5.0,0.0,4.0,45474a0a-ca40-42ed-9ab2-1ad008a834b5,False,45474a0a-ca40-42ed-9ab2-1ad008a834b5,45474a0a-ca40-42ed-9ab2-1ad008a834b5,0056T000009MdzFQAS,CR - Baked Goods,CR - Cookies,CR - Baked Goods,5k-7k,Tier 1,None,5736.0,2500.0
2,2022-10-01,Mexico,45474a0a-ca40-42ed-9ab2-1ad008a834b5,b9545910-bca6-4b2a-a603-363721e7a23d,MEX - Santa Fe,True,211,3,0,3,207,47727.46,330.00,2,1,1,2.483412,4.291667,0:03:40.696,3.130435,201,Stuffed,3.666667,2023-08-22 14:59:32.775832-05:00,0.014218,0.000000,0.014218,0.981043,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",Stuffed,0065G00000nahZ7QAI,Stuffed - MEX - Santa Fe,MEX - Santa Fe,CR - Baked Goods,0065G00000nahZ7QAI,Mexico City,Mexico,True,2022-12-31,10.0,1500.0,0.0,1000.0,2022-02-24,2022-03-24,2022-03-24,None,None,None,False,False,None,5.0,0.0,4.0,45474a0a-ca40-42ed-9ab2-1ad008a834b5,False,45474a0a-ca40-42ed-9ab2-1ad008a834b5,45474a0a-ca40-42ed-9ab2-1ad008a834b5,0056T000009MdzFQAS,CR - Baked Goods,CR - Cookies,CR - Baked Goods,5k-7k,Tier 1,None,5736.0,2500.0
3,2022-11-01,Mexico,45474a0a-ca40-42ed-9ab2-1ad008a834b5,864e0a3c-ecba-4305-b3ff-f22edd58c902,MEX - Polanco,True,178,2,2,4,174,39460.89,250.00,2,1,1,2.494382,4.666667,None,NaN,150,Stuffed,NaN,2023-08-22 14:59:32.775832-05:00,0.011236,0.011

**Calculating store health (GMV / MRR)**

**Importing and merging FX rates**

- MRR, GMV and health in USD

In [45]:
# Connect to google sheets exchange rate file
import pygsheets
from google.oauth2 import service_account
from ds.utilities.secrets import secrets
import pandas_gbq
import requests

In [46]:
# Connecting and authenticating with credentials

s = secrets.Secrets()
SCOPES = ('https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive', 'https://www.googleapis.com/auth/cloud-platform')
credentials = service_account.Credentials.from_service_account_info(s.get_secret("cloud_retail_sp_service_account"), scopes = SCOPES)

gc = pygsheets.authorize(custom_credentials=credentials)

# Importing worksheet
ex = gc.open_by_key('1h_lgOO2wf89txM2jX5ZXS64DizqewyPUReI_0-zP2M0')
exws = ex.worksheet('title','latam_monthly')
latam_monthly = exws.get_as_df(has_header=True,start="A1", end=None,empty_value='')

In [47]:
# Inspecting
latam_monthly.head()

,created_month,country,currency_code,exchange_rate
0,2019-01-01,Mexico,MXN,18.8
1,2019-02-01,Mexico,MXN,19.59
2,2019-03-01,Mexico,MXN,22.26
3,2019-04-01,Mexico,MXN,24.2
4,2019-05-01,Mexico,MXN,23.47


In [48]:
# Replacing NAs with 0 to enable conversion to numeric
latam_monthly['exchange_rate']= latam_monthly['exchange_rate'].replace('#N/A','0')
latam_monthly['exchange_rate']= pd.to_numeric(latam_monthly['exchange_rate'], errors='coerce')

In [49]:
# Merging FX rates to combined data
combined_data_fx = combined_data.merge(latam_monthly, how = "left", on = ['created_month','country'])

In [50]:
# Calculating GMV, MRR and Health in USD
combined_data_fx["mrr_usd"] = combined_data_fx["MRR_local"] / combined_data_fx["exchange_rate"]
combined_data_fx["total_gmv_usd"] = combined_data_fx["total_gmv_local"] / combined_data_fx["exchange_rate"]
combined_data_fx["cancelled_gmv_usd"] = combined_data_fx["cancelled_gmv_local"] / combined_data_fx["exchange_rate"]
combined_data_fx["succesful_gmv_usd"] = combined_data_fx["total_gmv_usd"] - combined_data_fx["cancelled_gmv_usd"]
combined_data_fx["succesful_orders"] = combined_data_fx["total_orders"] - combined_data_fx["cancelled_orders"]

In [51]:
# Calculate total CU FT per opps
combined_data_fx["total_cu_ft"] = combined_data_fx["Storage_Amount_Dry_CU_FT__c"] + combined_data_fx["Storage_Amount_Frozen_CU_FT__c"] + combined_data_fx["Storage_Amount_Cold_CU_FT__c"]

In [52]:
# Examining data types
combined_data_fx.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6936 entries, 0 to 6935
Data columns (total 77 columns):
 #   Column                                   Non-Null Count  Dtype                         
---  ------                                   --------------  -----                         
 0   created_month                            6936 non-null   object                        
 1   country                                  6936 non-null   object                        
 2   organization_id                          6936 non-null   object                        
 3   facility_id                              6936 non-null   object                        
 4   facility_sf_name                         6936 non-null   object                        
 5   org_deleted_skusku                       6936 non-null   bool                          
 6   total_orders                             6936 non-null   int64                         
 7   cancelled_orders                         6936 non-n

**Calculating operations handling income**

- average processing fee % * succesful gmv

In [53]:
# Calculating income in local currency and USD
combined_data_fx["processing_fee_pct"] = combined_data_fx["Processing_Fee__c"] / 100
combined_data_fx["succesful_gmv_local"] = combined_data_fx["total_gmv_local"] - combined_data_fx["cancelled_gmv_local"]
combined_data_fx["processing_income_local"] = combined_data_fx["succesful_gmv_local"] * combined_data_fx["processing_fee_pct"]
combined_data_fx["processing_income_usd"] = combined_data_fx["succesful_gmv_usd"] * combined_data_fx["processing_fee_pct"]

In [54]:
# Create a concatenated field for name + facililty
combined_data_fx["name_facility"] = combined_data_fx["name"] + "-" + combined_data_fx["facility_sf_name"]

In [55]:
# Calculate org_health = gmv / mrr with local currency 
combined_data_fx["store_gmv_mrr"] = combined_data_fx["succesful_gmv_local"] / combined_data_fx["MRR_local"]

# Create org_health categories
health_cats = []
for i in combined_data_fx['store_gmv_mrr']:
    if i>=15:
        j='15x or more'
    elif i>=10:
        j='10x to 14.9x'
    elif i>=5:
        j='5x to 9.9x'
    elif i>=4:
        j='4x'
    elif i>=3:
        j='3x'
    elif i>=2:
        j='2x'
    elif i>=1:
        j='1x'
    else:
        j='0X'
    health_cats.append(j)

# Append categories
combined_data_fx["gmv_mrr_cat"] = health_cats

# Inspect
combined_data_fx.sample(3)

,created_month,country,organization_id,facility_id,facility_sf_name,org_deleted_skusku,total_orders,cancelled_orders,issue_orders,ODR_orders,scanned_orders,total_gmv_local,cancelled_gmv_local,OFOs,cities,facilities,Avg_Items_per_Order,avg_rating,avg_picking_time,avg_picking_time_minutes,orders_10picked,name,avg_picking_mins,data_last_updated,pct_cancelled,pct_issues,pct_odr,pct_scanned,attributes,Account,Account_Name_Jira__c,Opportunity_ID_18__c,Name,Facility_Name__c,Cuisine_Type__c,Id,Kitchen_City__c,Kitchen_Country__c,Is_Churned__c,Churn_Date__c,Processing_Fee__c,Monthly_Shelf_Fee_Dry_Storage__c,Monthly_Shelf_Fee_Frozen_Storage__c,Monthly_Shelf_Fee_Cold_Storage__c,Closed_Won_Date__c,Move_In_Date__c,Start_Date__c,Loss_Reason__c,Specific_Lost_Reason__c,Lost_Explanation__c,CR_Opp_Opted_out_Affiliate__c,CR_Opp_Restrictions_Affiliate__c,CR_Opp_Type_of_Processing_Fee__c,Storage_Amount_Dry_CU_FT__c,Storage_Amount_Frozen_CU_FT__c,Storage_Amount_Cold_CU_FT__c,sfdc_org_id,current_customer,org_id,Org_Id_Otter__c,CSM__c,CR_Account_Categories__c,CR_Account_Sub_category__c,CR_Account_Cuisine_Type__c,CR_Acc_Social_Media_Presence__c,LAM_TierScrape__c,CR_Type_of_Customer__c,CR_Acc_Social_Media_Presence_Numeric__c,MRR_local,currency_code,exchange_rate,mrr_usd,total_gmv_usd,cancelled_gmv_usd,succesful_gmv_usd,succesful_orders,total_cu_ft,processing_fee_pct,succesful_gmv_local,processing_income_local,processing_income_usd,name_facility,store_gmv_mrr,gmv_mrr_cat
5415,2023-03-01,Mexico,da6b5215-b442-4581-a2cb-210bf1a27a74,bcb04949-3cda-4ca0-be39-b9f579e356ca,MEX - Roma,False,20,0,1,1,20,10522.00,0.0,1,1,1,4.500000,5.0,None,NaN,17,Helarte Gelatos,NaN,2023-08-22 14:59:32.775832-05:00,0.000000,0.05,0.050000,1.000000,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",Helarte Gelatos,0065G00000aE2lxQAC,Helarte Gelatos - MEX - Roma,MEX - Roma,"CR - Ice Cream, Popsicle, Cookie Dough & Gelato",0065G00000aE2lxQAC,Mexico City,Mexico,False,None,5.0,0.0,1520.0,0.0,2021-07-15,2021-07-26,2021-07-26,None,None,None,False,False,None,0.0,8.0,0.0,da6b5215-b442-4581-a2cb-210bf1a27a74,False,da6b5215-b442-4581-a2cb-210bf1a27a74,da6b5215-b442-4581-a2cb-210bf1a27a74,0055G0000080B5LQAU,CR - Frozen Desserts,CR - Gelato,"CR - Ice Cream, Popsicle, Cookie Dough & Gelato",1k-3k,Tier 2,None,1599.0,1520.0,MXN,18.41,82.563824,571.537208,0.000000,571.537208,20,8.0,0.05,10522.00,526.1000,28.576860,Helarte Gelatos-MEX - Roma,6.922368,5x to 9.9x
3615,2022-08-01,Mexico,11ef0598-b249-45fc-9127-22afff938b4c,47e2d054-8b82-420c-b1b0-18010f57489c,MEX - Insurgentes Sur,False,3,0,0,0,3,5225.00,0.0,1,1,1,2.666667,NaN,00:05:49.500,-5.500000,1,Monte Xanic,5.816667,2023-08-22 14:59:32.775832-05:00,0.000000,0.00,0.000000,1.000000,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",Monte Xanic,0065G00000j6WOyQAM,Monte Xanic - MEX - Insurgentes Sur,MEX - Insurgentes Sur,CR - Alcoholic Beverages,0065G00000j6WOyQAM,Mexico City,Mexico,True,2022-08-31,0.0,2500.0,0.0,0.0,2021-12-07,2022-01-15,2022-01-15,None,None,None,False,False,None,10.0,0.0,0.0,11ef0598-b249-45fc-9127-22afff938b4c,False,11ef0598-b249-45fc-9127-22afff938b4c,11ef0598-b249-45fc-9127-22afff938b4c,0055G000006QIsHQAW,CR - Alcoholic Beverages,CR - Wine,CR - Alcoholic Beverages,50k-100k,Tier 1,None,83200.0,2500.0,MXN,20.13,124.192747,259.562842,0.000000,259.562842,3,10.0,0.00,5225.00,0.0000,0.000000,Monte Xanic-MEX - Insurgentes Sur,2.090000,2x
5273,2022-10-01,Brazil,238d0a94-e354-43f0-b095-6d37084cc126,87a40094-cf48-4032-8166-e4cee61bce2f,SAO - Dino,False,191,4,0,4,170,24989.86,628.5,1,1,1,1.424084,5.0,0:08:41.079,8.236842,135,Vide Vino,8.683333,2023-08-22 14:59:32.775832-05:00,0.020942,0.00,0.020942,0.890052,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",Vide Vino,0065G00000nbgCjQAI,Vide Vino - SAO - Dino,SAO - Dino,CR - Alcoholic Beverages,0065G00000nbgCjQAI,Sao Paulo,Brazil,Fal

In [56]:
combined_data_fx[combined_data_fx['name']=='Vape Mate'].head()

,created_month,country,organization_id,facility_id,facility_sf_name,org_deleted_skusku,total_orders,cancelled_orders,issue_orders,ODR_orders,scanned_orders,total_gmv_local,cancelled_gmv_local,OFOs,cities,facilities,Avg_Items_per_Order,avg_rating,avg_picking_time,avg_picking_time_minutes,orders_10picked,name,avg_picking_mins,data_last_updated,pct_cancelled,pct_issues,pct_odr,pct_scanned,attributes,Account,Account_Name_Jira__c,Opportunity_ID_18__c,Name,Facility_Name__c,Cuisine_Type__c,Id,Kitchen_City__c,Kitchen_Country__c,Is_Churned__c,Churn_Date__c,Processing_Fee__c,Monthly_Shelf_Fee_Dry_Storage__c,Monthly_Shelf_Fee_Frozen_Storage__c,Monthly_Shelf_Fee_Cold_Storage__c,Closed_Won_Date__c,Move_In_Date__c,Start_Date__c,Loss_Reason__c,Specific_Lost_Reason__c,Lost_Explanation__c,CR_Opp_Opted_out_Affiliate__c,CR_Opp_Restrictions_Affiliate__c,CR_Opp_Type_of_Processing_Fee__c,Storage_Amount_Dry_CU_FT__c,Storage_Amount_Frozen_CU_FT__c,Storage_Amount_Cold_CU_FT__c,sfdc_org_id,current_customer,org_id,Org_Id_Otter__c,CSM__c,CR_Account_Categories__c,CR_Account_Sub_category__c,CR_Account_Cuisine_Type__c,CR_Acc_Social_Media_Presence__c,LAM_TierScrape__c,CR_Type_of_Customer__c,CR_Acc_Social_Media_Presence_Numeric__c,MRR_local,currency_code,exchange_rate,mrr_usd,total_gmv_usd,cancelled_gmv_usd,succesful_gmv_usd,succesful_orders,total_cu_ft,processing_fee_pct,succesful_gmv_local,processing_income_local,processing_income_usd,name_facility,store_gmv_mrr,gmv_mrr_cat


In [57]:
# Drop unused columns
combined_data_fx = combined_data_fx.drop(columns = ["attributes", "Account", "sfdc_org_id", "org_id", "Facility_Name__c", "Id"])

# Inspect
combined_data_fx.head()

,created_month,country,organization_id,facility_id,facility_sf_name,org_deleted_skusku,total_orders,cancelled_orders,issue_orders,ODR_orders,scanned_orders,total_gmv_local,cancelled_gmv_local,OFOs,cities,facilities,Avg_Items_per_Order,avg_rating,avg_picking_time,avg_picking_time_minutes,orders_10picked,name,avg_picking_mins,data_last_updated,pct_cancelled,pct_issues,pct_odr,pct_scanned,Account_Name_Jira__c,Opportunity_ID_18__c,Name,Cuisine_Type__c,Kitchen_City__c,Kitchen_Country__c,Is_Churned__c,Churn_Date__c,Processing_Fee__c,Monthly_Shelf_Fee_Dry_Storage__c,Monthly_Shelf_Fee_Frozen_Storage__c,Monthly_Shelf_Fee_Cold_Storage__c,Closed_Won_Date__c,Move_In_Date__c,Start_Date__c,Loss_Reason__c,Specific_Lost_Reason__c,Lost_Explanation__c,CR_Opp_Opted_out_Affiliate__c,CR_Opp_Restrictions_Affiliate__c,CR_Opp_Type_of_Processing_Fee__c,Storage_Amount_Dry_CU_FT__c,Storage_Amount_Frozen_CU_FT__c,Storage_Amount_Cold_CU_FT__c,current_customer,Org_Id_Otter__c,CSM__c,CR_Account_Categories__c,CR_Account_Sub_category__c,CR_Account_Cuisine_Type__c,CR_Acc_Social_Media_Presence__c,LAM_TierScrape__c,CR_Type_of_Customer__c,CR_Acc_Social_Media_Presence_Numeric__c,MRR_local,currency_code,exchange_rate,mrr_usd,total_gmv_usd,cancelled_gmv_usd,succesful_gmv_usd,succesful_orders,total_cu_ft,processing_fee_pct,succesful_gmv_local,processing_income_local,processing_income_usd,name_facility,store_gmv_mrr,gmv_mrr_cat
0,2023-05-01,Brazil,e445cbbd-b698-43cf-9ccc-4362d07b4381,f7a65e9f-3342-48d2-a0fb-e9e4633c4bf6,RIO - Tijuca,False,435,13,0,13,424,17267.54,470.99,2,1,1,1.974713,4.803571,None,NaN,395,Brownice - Tijuca - Doces e Sobremesas - Pik n...,NaN,2023-08-22 14:59:32.775832-05:00,0.029885,0.000000,0.029885,0.974713,Brownice,0065G00000j7FmKQAU,Brownice - RIO - Tijuca,CR - Baked Goods,Rio de Janeiro,Brazil,False,None,5.0,930.0,0.0,1395.0,2021-12-28,2022-05-02,2022-05-02,None,None,None,False,False,None,8.0,0.0,12.0,False,e445cbbd-b698-43cf-9ccc-4362d07b4381,0056T000009cfwEQAQ,CR - Baked Goods,CR - Brownies,CR - Baked Goods,10k-20k,Tier 2,None,12000.0,2325.0,BRL,5.01,464.071856,3446.614770,94.009980,3352.604790,422,20.0,0.05,16796.55,839.8275,167.630240,Brownice - Tijuca - Doces e Sobremesas - Pik n...,7.224323,5x to 9.9x
1,2022-09-01,Mexico,45474a0a-ca40-42ed-9ab2-1ad008a834b5,b9545910-bca6-4b2a-a603-363721e7a23d,MEX - Santa Fe,True,211,6,1,7,204,49277.76,401.55,2,1,1,2.203791,4.466667,0:02:52.720,2.508475,194,Stuffed,2.866667,2023-08-22 14:59:32.775832-05:00,0.028436,0.004739,0.033175,0.966825,Stuffed,0065G00000nahZ7QAI,Stuffed - MEX - Santa Fe,CR - Baked Goods,Mexico City,Mexico,True,2022-12-31,10.0,1500.0,0.0,1000.0,2022-02-24,2022-03-24,2022-03-24,None,None,None,False,False,None,5.0,0.0,4.0,False,45474a0a-ca40-42ed-9ab2-1ad008a834b5,0056T000009MdzFQAS,CR - Baked Goods,CR - Cookies,CR - Baked Goods,5k-7k,Tier 1,None,5736.0,2500.0,MXN,20.06,124.626122,2456.518445,20.017448,2436.500997,205,9.0,0.10,48876.21,4887.6210,243.650100,Stuffed-MEX - Santa Fe,19.550484,15x or more
2,2022-10-01,Mexico,45474a0a-ca40-42ed-9ab2-1ad008a834b5,b9545910-bca6-4b2a-a603-363721e7a23d,MEX - Santa Fe,True,211,3,0,3,207,47727.46,330.00,2,1,1,2.483412,4.291667,0:03:40.696,3.130435,201,Stuffed,3.666667,2023-08-22 14:59:32.775832-05:00,0.014218,0.000000,0.014218,0.981043,Stuffed,0065G00000nahZ7QAI,Stuffed - MEX - Santa Fe,CR - Baked Goods,Mexico City,Mexico,True,2022-12-31,10.0,1500.0,0.0,1000.0,2022-02-24,2022-03-24,2022-03-24,None,None,None,False,False,None,5.0,0.0,4.0,False,45474a0a-ca40-42ed-9ab2-1ad008a834b5,0056T000009MdzFQAS,CR - Baked Goods,CR - Cookies,CR - Baked Goods,5k-7k,Tier 1,None,5736.0,2500.0,MXN,19.99,125.062531,2387.566783,16.508254,2371.058529,208,9.0,0.10,47397.46,4739.7460,237.105853,Stuffed-MEX - Santa Fe,18.958984,15x or more
3,2022-11-01,Mexico,45474a0a-ca40-42ed-9ab2-1ad008a834b5,864e0a3c-ecba-4305-b3ff-f22edd58c902,MEX - Polanco,True,178,2,2,4,174,39460.89,250.00,2,1,1,2.494382,4.666667,None,NaN,150,Stuffed,NaN,2023-08-22 14:59:32.775832-05:00,0.011236,0.01

In [58]:
# Renaming columns
combined_data_fx = combined_data_fx.rename(columns = {"facility_sf_name":"facility_name",
                                                      "organization_id": "org_id",
                                                      "Account_Name_Jira__c":"acct_name_jira",
                                                      "Kitchen_City__c":"kitchen_city",
                                                     "Kitchen_Country__c":"kitchen_country",
                                                     "Is_Churned__c":"is_churned",
                                                     "Churn_Date__c":"churn_date",
                                                     "Processing_Fee__c":"processing_fee_int",
                                                     "Closed_Won_Date__c":"closed_won_date"})

#Inspect
combined_data_fx.head()

,created_month,country,org_id,facility_id,facility_name,org_deleted_skusku,total_orders,cancelled_orders,issue_orders,ODR_orders,scanned_orders,total_gmv_local,cancelled_gmv_local,OFOs,cities,facilities,Avg_Items_per_Order,avg_rating,avg_picking_time,avg_picking_time_minutes,orders_10picked,name,avg_picking_mins,data_last_updated,pct_cancelled,pct_issues,pct_odr,pct_scanned,acct_name_jira,Opportunity_ID_18__c,Name,Cuisine_Type__c,kitchen_city,kitchen_country,is_churned,churn_date,processing_fee_int,Monthly_Shelf_Fee_Dry_Storage__c,Monthly_Shelf_Fee_Frozen_Storage__c,Monthly_Shelf_Fee_Cold_Storage__c,closed_won_date,Move_In_Date__c,Start_Date__c,Loss_Reason__c,Specific_Lost_Reason__c,Lost_Explanation__c,CR_Opp_Opted_out_Affiliate__c,CR_Opp_Restrictions_Affiliate__c,CR_Opp_Type_of_Processing_Fee__c,Storage_Amount_Dry_CU_FT__c,Storage_Amount_Frozen_CU_FT__c,Storage_Amount_Cold_CU_FT__c,current_customer,Org_Id_Otter__c,CSM__c,CR_Account_Categories__c,CR_Account_Sub_category__c,CR_Account_Cuisine_Type__c,CR_Acc_Social_Media_Presence__c,LAM_TierScrape__c,CR_Type_of_Customer__c,CR_Acc_Social_Media_Presence_Numeric__c,MRR_local,currency_code,exchange_rate,mrr_usd,total_gmv_usd,cancelled_gmv_usd,succesful_gmv_usd,succesful_orders,total_cu_ft,processing_fee_pct,succesful_gmv_local,processing_income_local,processing_income_usd,name_facility,store_gmv_mrr,gmv_mrr_cat
0,2023-05-01,Brazil,e445cbbd-b698-43cf-9ccc-4362d07b4381,f7a65e9f-3342-48d2-a0fb-e9e4633c4bf6,RIO - Tijuca,False,435,13,0,13,424,17267.54,470.99,2,1,1,1.974713,4.803571,None,NaN,395,Brownice - Tijuca - Doces e Sobremesas - Pik n...,NaN,2023-08-22 14:59:32.775832-05:00,0.029885,0.000000,0.029885,0.974713,Brownice,0065G00000j7FmKQAU,Brownice - RIO - Tijuca,CR - Baked Goods,Rio de Janeiro,Brazil,False,None,5.0,930.0,0.0,1395.0,2021-12-28,2022-05-02,2022-05-02,None,None,None,False,False,None,8.0,0.0,12.0,False,e445cbbd-b698-43cf-9ccc-4362d07b4381,0056T000009cfwEQAQ,CR - Baked Goods,CR - Brownies,CR - Baked Goods,10k-20k,Tier 2,None,12000.0,2325.0,BRL,5.01,464.071856,3446.614770,94.009980,3352.604790,422,20.0,0.05,16796.55,839.8275,167.630240,Brownice - Tijuca - Doces e Sobremesas - Pik n...,7.224323,5x to 9.9x
1,2022-09-01,Mexico,45474a0a-ca40-42ed-9ab2-1ad008a834b5,b9545910-bca6-4b2a-a603-363721e7a23d,MEX - Santa Fe,True,211,6,1,7,204,49277.76,401.55,2,1,1,2.203791,4.466667,0:02:52.720,2.508475,194,Stuffed,2.866667,2023-08-22 14:59:32.775832-05:00,0.028436,0.004739,0.033175,0.966825,Stuffed,0065G00000nahZ7QAI,Stuffed - MEX - Santa Fe,CR - Baked Goods,Mexico City,Mexico,True,2022-12-31,10.0,1500.0,0.0,1000.0,2022-02-24,2022-03-24,2022-03-24,None,None,None,False,False,None,5.0,0.0,4.0,False,45474a0a-ca40-42ed-9ab2-1ad008a834b5,0056T000009MdzFQAS,CR - Baked Goods,CR - Cookies,CR - Baked Goods,5k-7k,Tier 1,None,5736.0,2500.0,MXN,20.06,124.626122,2456.518445,20.017448,2436.500997,205,9.0,0.10,48876.21,4887.6210,243.650100,Stuffed-MEX - Santa Fe,19.550484,15x or more
2,2022-10-01,Mexico,45474a0a-ca40-42ed-9ab2-1ad008a834b5,b9545910-bca6-4b2a-a603-363721e7a23d,MEX - Santa Fe,True,211,3,0,3,207,47727.46,330.00,2,1,1,2.483412,4.291667,0:03:40.696,3.130435,201,Stuffed,3.666667,2023-08-22 14:59:32.775832-05:00,0.014218,0.000000,0.014218,0.981043,Stuffed,0065G00000nahZ7QAI,Stuffed - MEX - Santa Fe,CR - Baked Goods,Mexico City,Mexico,True,2022-12-31,10.0,1500.0,0.0,1000.0,2022-02-24,2022-03-24,2022-03-24,None,None,None,False,False,None,5.0,0.0,4.0,False,45474a0a-ca40-42ed-9ab2-1ad008a834b5,0056T000009MdzFQAS,CR - Baked Goods,CR - Cookies,CR - Baked Goods,5k-7k,Tier 1,None,5736.0,2500.0,MXN,19.99,125.062531,2387.566783,16.508254,2371.058529,208,9.0,0.10,47397.46,4739.7460,237.105853,Stuffed-MEX - Santa Fe,18.958984,15x or more
3,2022-11-01,Mexico,45474a0a-ca40-42ed-9ab2-1ad008a834b5,864e0a3c-ecba-4305-b3ff-f22edd58c902,MEX - Polanco,True,178,2,2,4,174,39460.89,250.00,2,1,1,2.494382,4.666667,None,NaN,150,Stuffed,NaN,2023-08-22 14:59:32.775832-05:00,0.011236,0.011236,0.022472,0.977528,Stuffed,0

In [59]:
# Filling NaNs with 0 on numeric columns
na_cols_gmv = ["MRR_local", "mrr_usd", "succesful_gmv_usd", "store_gmv_mrr"]

# Replacing NaNs
combined_data_fx[na_cols_gmv] = combined_data_fx[na_cols_gmv].fillna(0)

In [60]:
# Replacing inf values
combined_data_fx.replace([np.inf, -np.inf], 0, inplace=True)

In [61]:
combined_data_fx.head()

,created_month,country,org_id,facility_id,facility_name,org_deleted_skusku,total_orders,cancelled_orders,issue_orders,ODR_orders,scanned_orders,total_gmv_local,cancelled_gmv_local,OFOs,cities,facilities,Avg_Items_per_Order,avg_rating,avg_picking_time,avg_picking_time_minutes,orders_10picked,name,avg_picking_mins,data_last_updated,pct_cancelled,pct_issues,pct_odr,pct_scanned,acct_name_jira,Opportunity_ID_18__c,Name,Cuisine_Type__c,kitchen_city,kitchen_country,is_churned,churn_date,processing_fee_int,Monthly_Shelf_Fee_Dry_Storage__c,Monthly_Shelf_Fee_Frozen_Storage__c,Monthly_Shelf_Fee_Cold_Storage__c,closed_won_date,Move_In_Date__c,Start_Date__c,Loss_Reason__c,Specific_Lost_Reason__c,Lost_Explanation__c,CR_Opp_Opted_out_Affiliate__c,CR_Opp_Restrictions_Affiliate__c,CR_Opp_Type_of_Processing_Fee__c,Storage_Amount_Dry_CU_FT__c,Storage_Amount_Frozen_CU_FT__c,Storage_Amount_Cold_CU_FT__c,current_customer,Org_Id_Otter__c,CSM__c,CR_Account_Categories__c,CR_Account_Sub_category__c,CR_Account_Cuisine_Type__c,CR_Acc_Social_Media_Presence__c,LAM_TierScrape__c,CR_Type_of_Customer__c,CR_Acc_Social_Media_Presence_Numeric__c,MRR_local,currency_code,exchange_rate,mrr_usd,total_gmv_usd,cancelled_gmv_usd,succesful_gmv_usd,succesful_orders,total_cu_ft,processing_fee_pct,succesful_gmv_local,processing_income_local,processing_income_usd,name_facility,store_gmv_mrr,gmv_mrr_cat
0,2023-05-01,Brazil,e445cbbd-b698-43cf-9ccc-4362d07b4381,f7a65e9f-3342-48d2-a0fb-e9e4633c4bf6,RIO - Tijuca,False,435,13,0,13,424,17267.54,470.99,2,1,1,1.974713,4.803571,None,NaN,395,Brownice - Tijuca - Doces e Sobremesas - Pik n...,NaN,2023-08-22 14:59:32.775832-05:00,0.029885,0.000000,0.029885,0.974713,Brownice,0065G00000j7FmKQAU,Brownice - RIO - Tijuca,CR - Baked Goods,Rio de Janeiro,Brazil,False,None,5.0,930.0,0.0,1395.0,2021-12-28,2022-05-02,2022-05-02,NaN,None,None,False,False,None,8.0,0.0,12.0,False,e445cbbd-b698-43cf-9ccc-4362d07b4381,0056T000009cfwEQAQ,CR - Baked Goods,CR - Brownies,CR - Baked Goods,10k-20k,Tier 2,None,12000.0,2325.0,BRL,5.01,464.071856,3446.614770,94.009980,3352.604790,422,20.0,0.05,16796.55,839.8275,167.630240,Brownice - Tijuca - Doces e Sobremesas - Pik n...,7.224323,5x to 9.9x
1,2022-09-01,Mexico,45474a0a-ca40-42ed-9ab2-1ad008a834b5,b9545910-bca6-4b2a-a603-363721e7a23d,MEX - Santa Fe,True,211,6,1,7,204,49277.76,401.55,2,1,1,2.203791,4.466667,0:02:52.720,2.508475,194,Stuffed,2.866667,2023-08-22 14:59:32.775832-05:00,0.028436,0.004739,0.033175,0.966825,Stuffed,0065G00000nahZ7QAI,Stuffed - MEX - Santa Fe,CR - Baked Goods,Mexico City,Mexico,True,2022-12-31,10.0,1500.0,0.0,1000.0,2022-02-24,2022-03-24,2022-03-24,NaN,None,None,False,False,None,5.0,0.0,4.0,False,45474a0a-ca40-42ed-9ab2-1ad008a834b5,0056T000009MdzFQAS,CR - Baked Goods,CR - Cookies,CR - Baked Goods,5k-7k,Tier 1,None,5736.0,2500.0,MXN,20.06,124.626122,2456.518445,20.017448,2436.500997,205,9.0,0.10,48876.21,4887.6210,243.650100,Stuffed-MEX - Santa Fe,19.550484,15x or more
2,2022-10-01,Mexico,45474a0a-ca40-42ed-9ab2-1ad008a834b5,b9545910-bca6-4b2a-a603-363721e7a23d,MEX - Santa Fe,True,211,3,0,3,207,47727.46,330.00,2,1,1,2.483412,4.291667,0:03:40.696,3.130435,201,Stuffed,3.666667,2023-08-22 14:59:32.775832-05:00,0.014218,0.000000,0.014218,0.981043,Stuffed,0065G00000nahZ7QAI,Stuffed - MEX - Santa Fe,CR - Baked Goods,Mexico City,Mexico,True,2022-12-31,10.0,1500.0,0.0,1000.0,2022-02-24,2022-03-24,2022-03-24,NaN,None,None,False,False,None,5.0,0.0,4.0,False,45474a0a-ca40-42ed-9ab2-1ad008a834b5,0056T000009MdzFQAS,CR - Baked Goods,CR - Cookies,CR - Baked Goods,5k-7k,Tier 1,None,5736.0,2500.0,MXN,19.99,125.062531,2387.566783,16.508254,2371.058529,208,9.0,0.10,47397.46,4739.7460,237.105853,Stuffed-MEX - Santa Fe,18.958984,15x or more
3,2022-11-01,Mexico,45474a0a-ca40-42ed-9ab2-1ad008a834b5,864e0a3c-ecba-4305-b3ff-f22edd58c902,MEX - Polanco,True,178,2,2,4,174,39460.89,250.00,2,1,1,2.494382,4.666667,None,NaN,150,Stuffed,NaN,2023-08-22 14:59:32.775832-05:00,0.011236,0.011236,0.022472,0.977528,Stuffed,0065

In [62]:
# Drop unused columns
combined_data_fx = combined_data_fx.drop(columns = ["Name"])

# Inspect
combined_data_fx.head()

,created_month,country,org_id,facility_id,facility_name,org_deleted_skusku,total_orders,cancelled_orders,issue_orders,ODR_orders,scanned_orders,total_gmv_local,cancelled_gmv_local,OFOs,cities,facilities,Avg_Items_per_Order,avg_rating,avg_picking_time,avg_picking_time_minutes,orders_10picked,name,avg_picking_mins,data_last_updated,pct_cancelled,pct_issues,pct_odr,pct_scanned,acct_name_jira,Opportunity_ID_18__c,Cuisine_Type__c,kitchen_city,kitchen_country,is_churned,churn_date,processing_fee_int,Monthly_Shelf_Fee_Dry_Storage__c,Monthly_Shelf_Fee_Frozen_Storage__c,Monthly_Shelf_Fee_Cold_Storage__c,closed_won_date,Move_In_Date__c,Start_Date__c,Loss_Reason__c,Specific_Lost_Reason__c,Lost_Explanation__c,CR_Opp_Opted_out_Affiliate__c,CR_Opp_Restrictions_Affiliate__c,CR_Opp_Type_of_Processing_Fee__c,Storage_Amount_Dry_CU_FT__c,Storage_Amount_Frozen_CU_FT__c,Storage_Amount_Cold_CU_FT__c,current_customer,Org_Id_Otter__c,CSM__c,CR_Account_Categories__c,CR_Account_Sub_category__c,CR_Account_Cuisine_Type__c,CR_Acc_Social_Media_Presence__c,LAM_TierScrape__c,CR_Type_of_Customer__c,CR_Acc_Social_Media_Presence_Numeric__c,MRR_local,currency_code,exchange_rate,mrr_usd,total_gmv_usd,cancelled_gmv_usd,succesful_gmv_usd,succesful_orders,total_cu_ft,processing_fee_pct,succesful_gmv_local,processing_income_local,processing_income_usd,name_facility,store_gmv_mrr,gmv_mrr_cat
0,2023-05-01,Brazil,e445cbbd-b698-43cf-9ccc-4362d07b4381,f7a65e9f-3342-48d2-a0fb-e9e4633c4bf6,RIO - Tijuca,False,435,13,0,13,424,17267.54,470.99,2,1,1,1.974713,4.803571,None,NaN,395,Brownice - Tijuca - Doces e Sobremesas - Pik n...,NaN,2023-08-22 14:59:32.775832-05:00,0.029885,0.000000,0.029885,0.974713,Brownice,0065G00000j7FmKQAU,CR - Baked Goods,Rio de Janeiro,Brazil,False,None,5.0,930.0,0.0,1395.0,2021-12-28,2022-05-02,2022-05-02,NaN,None,None,False,False,None,8.0,0.0,12.0,False,e445cbbd-b698-43cf-9ccc-4362d07b4381,0056T000009cfwEQAQ,CR - Baked Goods,CR - Brownies,CR - Baked Goods,10k-20k,Tier 2,None,12000.0,2325.0,BRL,5.01,464.071856,3446.614770,94.009980,3352.604790,422,20.0,0.05,16796.55,839.8275,167.630240,Brownice - Tijuca - Doces e Sobremesas - Pik n...,7.224323,5x to 9.9x
1,2022-09-01,Mexico,45474a0a-ca40-42ed-9ab2-1ad008a834b5,b9545910-bca6-4b2a-a603-363721e7a23d,MEX - Santa Fe,True,211,6,1,7,204,49277.76,401.55,2,1,1,2.203791,4.466667,0:02:52.720,2.508475,194,Stuffed,2.866667,2023-08-22 14:59:32.775832-05:00,0.028436,0.004739,0.033175,0.966825,Stuffed,0065G00000nahZ7QAI,CR - Baked Goods,Mexico City,Mexico,True,2022-12-31,10.0,1500.0,0.0,1000.0,2022-02-24,2022-03-24,2022-03-24,NaN,None,None,False,False,None,5.0,0.0,4.0,False,45474a0a-ca40-42ed-9ab2-1ad008a834b5,0056T000009MdzFQAS,CR - Baked Goods,CR - Cookies,CR - Baked Goods,5k-7k,Tier 1,None,5736.0,2500.0,MXN,20.06,124.626122,2456.518445,20.017448,2436.500997,205,9.0,0.10,48876.21,4887.6210,243.650100,Stuffed-MEX - Santa Fe,19.550484,15x or more
2,2022-10-01,Mexico,45474a0a-ca40-42ed-9ab2-1ad008a834b5,b9545910-bca6-4b2a-a603-363721e7a23d,MEX - Santa Fe,True,211,3,0,3,207,47727.46,330.00,2,1,1,2.483412,4.291667,0:03:40.696,3.130435,201,Stuffed,3.666667,2023-08-22 14:59:32.775832-05:00,0.014218,0.000000,0.014218,0.981043,Stuffed,0065G00000nahZ7QAI,CR - Baked Goods,Mexico City,Mexico,True,2022-12-31,10.0,1500.0,0.0,1000.0,2022-02-24,2022-03-24,2022-03-24,NaN,None,None,False,False,None,5.0,0.0,4.0,False,45474a0a-ca40-42ed-9ab2-1ad008a834b5,0056T000009MdzFQAS,CR - Baked Goods,CR - Cookies,CR - Baked Goods,5k-7k,Tier 1,None,5736.0,2500.0,MXN,19.99,125.062531,2387.566783,16.508254,2371.058529,208,9.0,0.10,47397.46,4739.7460,237.105853,Stuffed-MEX - Santa Fe,18.958984,15x or more
3,2022-11-01,Mexico,45474a0a-ca40-42ed-9ab2-1ad008a834b5,864e0a3c-ecba-4305-b3ff-f22edd58c902,MEX - Polanco,True,178,2,2,4,174,39460.89,250.00,2,1,1,2.494382,4.666667,None,NaN,150,Stuffed,NaN,2023-08-22 14:59:32.775832-05:00,0.011236,0.011236,0.022472,0.977528,Stuffed,0065G00000nahYnQAI,CR - Baked Goods,Mexico City,Mexico,True,2022-12-31,10.0,1500.0,

In [63]:
combined_data_fx["total_gmv_usd"].sum()

8151185.412693219

In [64]:
# Converting corresponding columns to numeric and datetime
combined_data_fx["created_month"] = pd.to_datetime(combined_data_fx["created_month"])
combined_data_fx["total_orders"] = pd.to_numeric(combined_data_fx["total_orders"])
combined_data_fx["cancelled_orders"] = pd.to_numeric(combined_data_fx["cancelled_orders"])
combined_data_fx["issue_orders"] = pd.to_numeric(combined_data_fx["issue_orders"])
combined_data_fx["ODR_orders"] = pd.to_numeric(combined_data_fx["ODR_orders"])
combined_data_fx["scanned_orders"] = pd.to_numeric(combined_data_fx["scanned_orders"])
combined_data_fx["churn_date"] = pd.to_datetime(combined_data_fx["churn_date"])

In [65]:
# Week test
copy_test = combined_data_fx.copy()


In [66]:
copy_test.head(2)

,created_month,country,org_id,facility_id,facility_name,org_deleted_skusku,total_orders,cancelled_orders,issue_orders,ODR_orders,scanned_orders,total_gmv_local,cancelled_gmv_local,OFOs,cities,facilities,Avg_Items_per_Order,avg_rating,avg_picking_time,avg_picking_time_minutes,orders_10picked,name,avg_picking_mins,data_last_updated,pct_cancelled,pct_issues,pct_odr,pct_scanned,acct_name_jira,Opportunity_ID_18__c,Cuisine_Type__c,kitchen_city,kitchen_country,is_churned,churn_date,processing_fee_int,Monthly_Shelf_Fee_Dry_Storage__c,Monthly_Shelf_Fee_Frozen_Storage__c,Monthly_Shelf_Fee_Cold_Storage__c,closed_won_date,Move_In_Date__c,Start_Date__c,Loss_Reason__c,Specific_Lost_Reason__c,Lost_Explanation__c,CR_Opp_Opted_out_Affiliate__c,CR_Opp_Restrictions_Affiliate__c,CR_Opp_Type_of_Processing_Fee__c,Storage_Amount_Dry_CU_FT__c,Storage_Amount_Frozen_CU_FT__c,Storage_Amount_Cold_CU_FT__c,current_customer,Org_Id_Otter__c,CSM__c,CR_Account_Categories__c,CR_Account_Sub_category__c,CR_Account_Cuisine_Type__c,CR_Acc_Social_Media_Presence__c,LAM_TierScrape__c,CR_Type_of_Customer__c,CR_Acc_Social_Media_Presence_Numeric__c,MRR_local,currency_code,exchange_rate,mrr_usd,total_gmv_usd,cancelled_gmv_usd,succesful_gmv_usd,succesful_orders,total_cu_ft,processing_fee_pct,succesful_gmv_local,processing_income_local,processing_income_usd,name_facility,store_gmv_mrr,gmv_mrr_cat
0,2023-05-01,Brazil,e445cbbd-b698-43cf-9ccc-4362d07b4381,f7a65e9f-3342-48d2-a0fb-e9e4633c4bf6,RIO - Tijuca,False,435,13,0,13,424,17267.54,470.99,2,1,1,1.974713,4.803571,None,NaN,395,Brownice - Tijuca - Doces e Sobremesas - Pik n...,NaN,2023-08-22 14:59:32.775832-05:00,0.029885,0.000000,0.029885,0.974713,Brownice,0065G00000j7FmKQAU,CR - Baked Goods,Rio de Janeiro,Brazil,False,NaT,5.0,930.0,0.0,1395.0,2021-12-28,2022-05-02,2022-05-02,NaN,None,None,False,False,None,8.0,0.0,12.0,False,e445cbbd-b698-43cf-9ccc-4362d07b4381,0056T000009cfwEQAQ,CR - Baked Goods,CR - Brownies,CR - Baked Goods,10k-20k,Tier 2,None,12000.0,2325.0,BRL,5.01,464.071856,3446.614770,94.009980,3352.604790,422,20.0,0.05,16796.55,839.8275,167.63024,Brownice - Tijuca - Doces e Sobremesas - Pik n...,7.224323,5x to 9.9x
1,2022-09-01,Mexico,45474a0a-ca40-42ed-9ab2-1ad008a834b5,b9545910-bca6-4b2a-a603-363721e7a23d,MEX - Santa Fe,True,211,6,1,7,204,49277.76,401.55,2,1,1,2.203791,4.466667,0:02:52.720,2.508475,194,Stuffed,2.866667,2023-08-22 14:59:32.775832-05:00,0.028436,0.004739,0.033175,0.966825,Stuffed,0065G00000nahZ7QAI,CR - Baked Goods,Mexico City,Mexico,True,2022-12-31,10.0,1500.0,0.0,1000.0,2022-02-24,2022-03-24,2022-03-24,NaN,None,None,False,False,None,5.0,0.0,4.0,False,45474a0a-ca40-42ed-9ab2-1ad008a834b5,0056T000009MdzFQAS,CR - Baked Goods,CR - Cookies,CR - Baked Goods,5k-7k,Tier 1,None,5736.0,2500.0,MXN,20.06,124.626122,2456.518445,20.017448,2436.500997,205,9.0,0.10,48876.21,4887.6210,243.65010,Stuffed-MEX - Santa Fe,19.550484,15x or more


In [67]:
copy_test["Start_Date__c_utc"] = pd.to_datetime(copy_test["Start_Date__c"], utc = True)

In [68]:
copy_test["churn_or_current_date"] = copy_test[["churn_date", "created_month"]].bfill(axis=1).iloc[:, 0]
copy_test["churn_or_current_date"] = pd.to_datetime(copy_test["churn_or_current_date"], utc = True)

In [69]:
copy_test["weeks_of_operation"] = np.ceil((copy_test.churn_or_current_date - copy_test.Start_Date__c_utc)/np.timedelta64(1, "W"))
copy_test["weeks_of_operation"] = copy_test["weeks_of_operation"].fillna(0)
copy_test["weeks_of_operation"] = copy_test["weeks_of_operation"].astype(int)

In [70]:
copy_test.sample(3)

,created_month,country,org_id,facility_id,facility_name,org_deleted_skusku,total_orders,cancelled_orders,issue_orders,ODR_orders,scanned_orders,total_gmv_local,cancelled_gmv_local,OFOs,cities,facilities,Avg_Items_per_Order,avg_rating,avg_picking_time,avg_picking_time_minutes,orders_10picked,name,avg_picking_mins,data_last_updated,pct_cancelled,pct_issues,pct_odr,pct_scanned,acct_name_jira,Opportunity_ID_18__c,Cuisine_Type__c,kitchen_city,kitchen_country,is_churned,churn_date,processing_fee_int,Monthly_Shelf_Fee_Dry_Storage__c,Monthly_Shelf_Fee_Frozen_Storage__c,Monthly_Shelf_Fee_Cold_Storage__c,closed_won_date,Move_In_Date__c,Start_Date__c,Loss_Reason__c,Specific_Lost_Reason__c,Lost_Explanation__c,CR_Opp_Opted_out_Affiliate__c,CR_Opp_Restrictions_Affiliate__c,CR_Opp_Type_of_Processing_Fee__c,Storage_Amount_Dry_CU_FT__c,Storage_Amount_Frozen_CU_FT__c,Storage_Amount_Cold_CU_FT__c,current_customer,Org_Id_Otter__c,CSM__c,CR_Account_Categories__c,CR_Account_Sub_category__c,CR_Account_Cuisine_Type__c,CR_Acc_Social_Media_Presence__c,LAM_TierScrape__c,CR_Type_of_Customer__c,CR_Acc_Social_Media_Presence_Numeric__c,MRR_local,currency_code,exchange_rate,mrr_usd,total_gmv_usd,cancelled_gmv_usd,succesful_gmv_usd,succesful_orders,total_cu_ft,processing_fee_pct,succesful_gmv_local,processing_income_local,processing_income_usd,name_facility,store_gmv_mrr,gmv_mrr_cat,Start_Date__c_utc,churn_or_current_date,weeks_of_operation
2071,2022-11-01,Colombia,2ae1d0a8-5caf-4692-bd72-07eb6cbdc77f,dd16bf0e-8a89-4f21-bc02-1c9793227502,BOG - San Felipe,False,33,5,0,5,21,1571600.00,404800.00,1,1,1,1.545455,3.666667,None,NaN,23,Jade Pimont Patisserie,NaN,2023-08-22 14:59:32.775832-05:00,0.151515,0.0,0.151515,0.636364,Jade Pimont Pâtisserie,0066T000018boxuQAA,CR - Baked Goods,Bogota,Colombia,True,2022-12-31,8.0,183334.0,183333.0,183333.00,2022-08-30,2022-10-05,2022-10-14,NaN,None,None,False,False,None,4.0,4.0,4.0,False,2ae1d0a8-5caf-4692-bd72-07eb6cbdc77f,None,CR - Baked Goods,CR - Cakes,CR - Baked Goods,10k-20k,Tier 3,Local,18700.0,550000.00,COP,4928.88,111.587217,318.855399,82.128191,236.727208,28,12.0,0.08,1166800.00,93344.0000,18.938177,Jade Pimont Patisserie-BOG - San Felipe,2.121455,2x,2022-10-14 00:00:00+00:00,2022-12-31 00:00:00+00:00,12
5493,2023-07-01,Brazil,b321d123-a955-4270-8bba-d8ebbbf31720,4518d99c-fa47-4da7-b5f5-c071c78b5ac2,BHZ - Tancredo Neves,False,59,3,0,3,53,2772.72,111.89,2,1,1,2.084746,1.000000,None,NaN,39,Petit Brigadier,NaN,2023-08-22 14:59:32.775832-05:00,0.050847,0.0,0.050847,0.898305,Petit Brigadier,0066T000018c3AgQAI,CR - Baked Goods,Belo Horizonte,Brazil,False,NaT,8.0,0.0,280.0,309.27,2022-08-22,2022-03-04,2021-12-27,NaN,None,None,False,False,None,0.0,4.0,4.0,False,b321d123-a955-4270-8bba-d8ebbbf31720,005f40000054nEXAAY,CR - Desserts,CR - Brigadeiro,CR - Baked Goods,5k-7k,Tier 3,None,5642.0,589.27,BRL,4.80,122.764583,577.650000,23.310417,554.339583,56,8.0,0.08,2660.83,212.8664,44.347167,Petit Brigadier-BHZ - Tancredo Neves,4.515468,4x,2021-12-27 00:00:00+00:00,2023-07-01 00:00:00+00:00,79
5847,2023-06-01,Brazil,e59be2c8-b884-4e16-905f-e67f87c93a59,f7a65e9f-3342-48d2-a0fb-e9e4633c4bf6,RIO - Tijuca,False,302,7,0,7,298,13641.11,285.00,1,1,1,1.801325,4.789474,None,NaN,259,Cucurucho Doceria - Pik n' Pak,NaN,2023-08-22 14:59:32.775832-05:00,0.023179,0.0,0.023179,0.986755,Cucurucho Doceria,0066T00001BQHb9QAH,None,Rio de Janeiro,Brazil,False,NaT,7.0,0.0,0.0,186.00,2023-03-31,2023-06-01,2023-06-01,NaN,None,None,False,False,Percentage / Order,0.0,0.0,4.0,True,e59be2c8-b884-4e16-905f-e67f87c93a59,0056T000009cfwEQAQ,CR - Desserts,CR - Pies,CR - Baked Goods,10k-20k,Tier 2,Local,20100.0,186.00,BRL,4.86,38.271605,2806.812757,58.641975,2748.170782,295,4.0,0.07,13356.11,934.9277,192.371955,Cucurucho Doceria - Pik n' Pak-RIO - Tijuca,71.807043,15x or more,2023-06-01 00:00:00+00:00,2023-06-01 00:00:00+00:00,0


In [71]:
copy_test["weeks_of_operation"].unique()

array([ 52,  41,  22,  11,  34,  44, 144, 161,  10,  -4,  39,  35,   9,
       100,  18,  20,   0,  36,  -1,  46,  -2,   5,  29,  13,   6,   4,
        21, 113,  30,  -3,  26,  48,   8,  66,  28,  96,  19,  70,  53,
        90,  56,  68,   7,  25,  86, 111, 150,  27,  61,   1,  49,  17,
       -28,  38,  14,  95,  23,  47,  83,  31, 122, 115,  92,  50,  87,
       105,  57,  -5,  24,  71,  15,  51,   2, -37,  62,  33,  80, 118,
        45,  74, 106,  55,  65,  16,  -7,  69, 131,  12, -41,  78, 135,
        42,  -8, 109, 139,  93,  -6,  81,   3,  60,  79, -11, 140, 166,
       -19, 137, 148,  54, 157, 126,  32,  98, -24,  75, 132,  64,  40,
       102,  73,  85, -15,  58, 110, 152, 127,  77, -32,  37, 124, 119,
        89, 128,  59, 108,  84,  63,  99, 116, 112, 151, 142, 120, 117,
       133,  67, 125, 147, 134, 103,  94, 130,  76,  82, 156,  43, 107,
       138,  72, -42, -45, -20, -13,  91, 121,  -9, 114, 101,  97, -33,
       -46, -23,  88, -22, -21, 129, 104, -43, -17, -26, -34, -3

In [72]:
copy_test[copy_test["weeks_of_operation"] == -2].sample()

,created_month,country,org_id,facility_id,facility_name,org_deleted_skusku,total_orders,cancelled_orders,issue_orders,ODR_orders,scanned_orders,total_gmv_local,cancelled_gmv_local,OFOs,cities,facilities,Avg_Items_per_Order,avg_rating,avg_picking_time,avg_picking_time_minutes,orders_10picked,name,avg_picking_mins,data_last_updated,pct_cancelled,pct_issues,pct_odr,pct_scanned,acct_name_jira,Opportunity_ID_18__c,Cuisine_Type__c,kitchen_city,kitchen_country,is_churned,churn_date,processing_fee_int,Monthly_Shelf_Fee_Dry_Storage__c,Monthly_Shelf_Fee_Frozen_Storage__c,Monthly_Shelf_Fee_Cold_Storage__c,closed_won_date,Move_In_Date__c,Start_Date__c,Loss_Reason__c,Specific_Lost_Reason__c,Lost_Explanation__c,CR_Opp_Opted_out_Affiliate__c,CR_Opp_Restrictions_Affiliate__c,CR_Opp_Type_of_Processing_Fee__c,Storage_Amount_Dry_CU_FT__c,Storage_Amount_Frozen_CU_FT__c,Storage_Amount_Cold_CU_FT__c,current_customer,Org_Id_Otter__c,CSM__c,CR_Account_Categories__c,CR_Account_Sub_category__c,CR_Account_Cuisine_Type__c,CR_Acc_Social_Media_Presence__c,LAM_TierScrape__c,CR_Type_of_Customer__c,CR_Acc_Social_Media_Presence_Numeric__c,MRR_local,currency_code,exchange_rate,mrr_usd,total_gmv_usd,cancelled_gmv_usd,succesful_gmv_usd,succesful_orders,total_cu_ft,processing_fee_pct,succesful_gmv_local,processing_income_local,processing_income_usd,name_facility,store_gmv_mrr,gmv_mrr_cat,Start_Date__c_utc,churn_or_current_date,weeks_of_operation
6678,2023-03-01,Brazil,852ddf9c-640c-472e-a38f-16c2e497c6c9,87a40094-cf48-4032-8166-e4cee61bce2f,SAO - Dino,False,17,1,0,1,16,1034.91,54.9,1,1,1,1.058824,5.0,None,NaN,8,El Pudim - Pik n' Pak,NaN,2023-08-22 14:59:32.775832-05:00,0.058824,0.0,0.058824,0.941176,El Pudim,0066T00001AtMnjQAF,None,Sao Paulo,Brazil,False,NaT,10.0,0.0,0.0,800.0,2023-02-14,2023-03-21,2023-03-21,NaN,None,None,True,False,None,0.0,0.0,8.0,False,852ddf9c-640c-472e-a38f-16c2e497c6c9,None,CR - Desserts,None,None,7k-10k,Tier 3,Local,7763.0,800.0,BRL,5.22,153.256705,198.258621,10.517241,187.741379,16,8.0,0.1,980.01,98.001,18.774138,El Pudim - Pik n' Pak-SAO - Dino,1.225012,1x,2023-03-21 00:00:00+00:00,2023-03-01 00:00:00+00:00,-2


In [73]:
tcc = copy_test[copy_test["name"] == "Tere Cazola"]
tcc.head()

,created_month,country,org_id,facility_id,facility_name,org_deleted_skusku,total_orders,cancelled_orders,issue_orders,ODR_orders,scanned_orders,total_gmv_local,cancelled_gmv_local,OFOs,cities,facilities,Avg_Items_per_Order,avg_rating,avg_picking_time,avg_picking_time_minutes,orders_10picked,name,avg_picking_mins,data_last_updated,pct_cancelled,pct_issues,pct_odr,pct_scanned,acct_name_jira,Opportunity_ID_18__c,Cuisine_Type__c,kitchen_city,kitchen_country,is_churned,churn_date,processing_fee_int,Monthly_Shelf_Fee_Dry_Storage__c,Monthly_Shelf_Fee_Frozen_Storage__c,Monthly_Shelf_Fee_Cold_Storage__c,closed_won_date,Move_In_Date__c,Start_Date__c,Loss_Reason__c,Specific_Lost_Reason__c,Lost_Explanation__c,CR_Opp_Opted_out_Affiliate__c,CR_Opp_Restrictions_Affiliate__c,CR_Opp_Type_of_Processing_Fee__c,Storage_Amount_Dry_CU_FT__c,Storage_Amount_Frozen_CU_FT__c,Storage_Amount_Cold_CU_FT__c,current_customer,Org_Id_Otter__c,CSM__c,CR_Account_Categories__c,CR_Account_Sub_category__c,CR_Account_Cuisine_Type__c,CR_Acc_Social_Media_Presence__c,LAM_TierScrape__c,CR_Type_of_Customer__c,CR_Acc_Social_Media_Presence_Numeric__c,MRR_local,currency_code,exchange_rate,mrr_usd,total_gmv_usd,cancelled_gmv_usd,succesful_gmv_usd,succesful_orders,total_cu_ft,processing_fee_pct,succesful_gmv_local,processing_income_local,processing_income_usd,name_facility,store_gmv_mrr,gmv_mrr_cat,Start_Date__c_utc,churn_or_current_date,weeks_of_operation
5264,2023-01-01,Mexico,b556b267-8821-4162-a0cc-254fc70dafa0,b9545910-bca6-4b2a-a603-363721e7a23d,MEX - Santa Fe,False,529,13,12,25,499,151218.60,5388.17,1,1,1,1.690566,4.754717,None,NaN,479,Tere Cazola,NaN,2023-08-22 14:59:32.775832-05:00,0.024575,0.022684,0.047259,0.943289,Tere Cazola,0065G00000j4xZhQAI,CR - Baked Goods,Mexico City,Mexico,True,2023-05-11,3.51,15812.5,0.0,0.0,2021-11-19,2021-10-06,2021-10-06,NaN,None,None,False,False,None,79.06,0.0,0.0,True,b556b267-8821-4162-a0cc-254fc70dafa0,005f4000006BISzAAO,CR - Baked Goods,CR - Cakes,CR - Baked Goods,10k-20k,Tier 1,None,47800.0,15812.5,MXN,18.98,833.113804,7967.260274,283.886723,7683.373551,516,79.06,0.0351,145830.43,5118.648093,269.686412,Tere Cazola-MEX - Santa Fe,9.222478,5x to 9.9x,2021-10-06 00:00:00+00:00,2023-05-11 00:00:00+00:00,84
5271,2022-09-01,Mexico,b556b267-8821-4162-a0cc-254fc70dafa0,b9545910-bca6-4b2a-a603-363721e7a23d,MEX - Santa Fe,False,271,5,4,9,264,69867.45,1568.10,1,1,1,1.630996,4.484375,0:03:49.974,3.323308,251,Tere Cazola,3.816667,2023-08-22 14:59:32.775832-05:00,0.018450,0.014760,0.033210,0.974170,Tere Cazola,0065G00000j4xZhQAI,CR - Baked Goods,Mexico City,Mexico,True,2023-05-11,3.51,15812.5,0.0,0.0,2021-11-19,2021-10-06,2021-10-06,NaN,None,None,False,False,None,79.06,0.0,0.0,True,b556b267-8821-4162-a0cc-254fc70dafa0,005f4000006BISzAAO,CR - Baked Goods,CR - Cakes,CR - Baked Goods,10k-20k,Tier 1,None,47800.0,15812.5,MXN,20.06,788.260219,3482.923729,78.170489,3404.753240,266,79.06,0.0351,68299.35,2397.307185,119.506839,Tere Cazola-MEX - Santa Fe,4.319326,4x,2021-10-06 00:00:00+00:00,2023-05-11 00:00:00+00:00,84
5274,2022-12-01,Mexico,b556b267-8821-4162-a0cc-254fc70dafa0,47e2d054-8b82-420c-b1b0-18010f57489c,MEX - Insurgentes Sur,False,503,11,12,23,484,151114.31,4390.62,1,1,1,1.749503,4.625000,None,NaN,455,Tere Cazola,NaN,2023-08-22 14:59:32.775832-05:00,0.021869,0.023857,0.045726,0.962227,Tere Cazola,0065G00000j4xWtQAI,CR - Baked Goods,Mexico City,Mexico,True,2023-05-11,3.51,15812.5,0.0,0.0,2021-11-19,2021-11-01,2021-11-01,NaN,None,None,False,False,Monthly Fixed Fee,79.06,0.0,0.0,True,b556b267-8821-4162-a0cc-254fc70dafa0,005f4000006BISzAAO,CR - Baked Goods,CR - Cakes,CR - Baked Goods,10k-20k,Tier 1,None,47800.0,15812.5,MXN,19.59,807.172027,7713.849413,224.125574,7489.723839,492,79.06,0.0351,146723.69,5150.001519,262.889307,Tere Cazola-MEX - Insurgentes Sur,9.278969,5x to 9.9x,2021-11-01 00:00:00+00:00,2023-05-11 00:00:00+00:00,80
5284,2022-10-01,Mexico,b556b267-8821-4162-a0cc-254fc70dafa0,9101d4f9-828f-45c5-b8b6-16986e76b374,MEX - Pedregal,False

In [74]:
tcc["weeks_of_operation"].unique()

array([84, 80, 52, 92, 91, 66, 61, 78, 39, 44, 53, 35, 65, 57, 73, 82, 70,
       48, 87, 43, 95, 74, 69, 56, 83])

**Uploading data to Google Sheets and Google Big Query**

**Renaming columns to new naming convention**

Nov. 2022

In [75]:
cols = list(combined_data_fx.columns.values)
cols

['created_month',
 'country',
 'org_id',
 'facility_id',
 'facility_name',
 'org_deleted_skusku',
 'total_orders',
 'cancelled_orders',
 'issue_orders',
 'ODR_orders',
 'scanned_orders',
 'total_gmv_local',
 'cancelled_gmv_local',
 'OFOs',
 'cities',
 'facilities',
 'Avg_Items_per_Order',
 'avg_rating',
 'avg_picking_time',
 'avg_picking_time_minutes',
 'orders_10picked',
 'name',
 'avg_picking_mins',
 'data_last_updated',
 'pct_cancelled',
 'pct_issues',
 'pct_odr',
 'pct_scanned',
 'acct_name_jira',
 'Opportunity_ID_18__c',
 'Cuisine_Type__c',
 'kitchen_city',
 'kitchen_country',
 'is_churned',
 'churn_date',
 'processing_fee_int',
 'Monthly_Shelf_Fee_Dry_Storage__c',
 'Monthly_Shelf_Fee_Frozen_Storage__c',
 'Monthly_Shelf_Fee_Cold_Storage__c',
 'closed_won_date',
 'Move_In_Date__c',
 'Start_Date__c',
 'Loss_Reason__c',
 'Specific_Lost_Reason__c',
 'Lost_Explanation__c',
 'CR_Opp_Opted_out_Affiliate__c',
 'CR_Opp_Restrictions_Affiliate__c',
 'CR_Opp_Type_of_Processing_Fee__c',
 'Stor

In [76]:
# Selecting order to avoid affecting current MBR worksheet formulas
print(len(combined_data_fx))

# Order
combined_data_fx = combined_data_fx[['created_month',
                                     'country',
                                     'org_id',
                                     'facility_id',
                                     'facility_name',
                                     'org_deleted_skusku',
                                     'total_orders',
                                     'cancelled_orders',
                                     'issue_orders',
                                     'ODR_orders',
                                     'scanned_orders',
                                     'total_gmv_local',
                                     'cancelled_gmv_local',
                                     'OFOs',
                                     'cities',
                                     'facilities',
                                     'Avg_Items_per_Order',
                                     'avg_rating',
                                     'avg_picking_time',
                                     'avg_picking_time_minutes',
                                     'name',
                                     'avg_picking_mins',
                                     'data_last_updated',
                                     'pct_cancelled',
                                     'pct_issues',
                                     'pct_odr',
                                     'acct_name_jira',
                                     'Opportunity_ID_18__c',
                                     'Cuisine_Type__c',
                                     'kitchen_city',
                                     'kitchen_country',
                                     'is_churned',
                                     'churn_date',
                                     'processing_fee_int',
                                     'Monthly_Shelf_Fee_Dry_Storage__c',
                                     'Monthly_Shelf_Fee_Frozen_Storage__c',
                                     'Monthly_Shelf_Fee_Cold_Storage__c',
                                     'closed_won_date',
                                     'Move_In_Date__c',
                                     'Start_Date__c',
                                     'Loss_Reason__c',
                                     'Specific_Lost_Reason__c',
                                     'Lost_Explanation__c',
                                     'CR_Opp_Opted_out_Affiliate__c',
                                     'CR_Opp_Restrictions_Affiliate__c',
                                     'CR_Opp_Type_of_Processing_Fee__c',
                                     'current_customer',
                                     #'Org_Id_Otter__c',  
                                     'MRR_local',
                                     'currency_code',
                                     'exchange_rate',
                                     'mrr_usd',
                                     'total_gmv_usd',
                                     'cancelled_gmv_usd',
                                     'succesful_gmv_usd',
                                     'processing_fee_pct',
                                     'succesful_gmv_local',
                                     'processing_income_local',
                                     'processing_income_usd',
                                     'name_facility',
                                     'store_gmv_mrr',
                                     'gmv_mrr_cat',
                                     'succesful_orders',
                                     'total_cu_ft',
                                     'pct_scanned',
                                     'CSM__c',
                                     'CR_Account_Categories__c',
                                     'CR_Account_Sub_category__c',
                                     'CR_Account_Cuisine_Type__c',
                                     'CR_Acc_Social_Media_Presence__c',
                                     'LAM_TierScrape__c',
                                     'CR_Type_of_Customer__c',
                                     'orders_10picked',
                                     'CR_Acc_Social_Media_Presence_Numeric__c']]

6936


In [77]:
# Inspect
combined_data_fx.head(2)

,created_month,country,org_id,facility_id,facility_name,org_deleted_skusku,total_orders,cancelled_orders,issue_orders,ODR_orders,scanned_orders,total_gmv_local,cancelled_gmv_local,OFOs,cities,facilities,Avg_Items_per_Order,avg_rating,avg_picking_time,avg_picking_time_minutes,name,avg_picking_mins,data_last_updated,pct_cancelled,pct_issues,pct_odr,acct_name_jira,Opportunity_ID_18__c,Cuisine_Type__c,kitchen_city,kitchen_country,is_churned,churn_date,processing_fee_int,Monthly_Shelf_Fee_Dry_Storage__c,Monthly_Shelf_Fee_Frozen_Storage__c,Monthly_Shelf_Fee_Cold_Storage__c,closed_won_date,Move_In_Date__c,Start_Date__c,Loss_Reason__c,Specific_Lost_Reason__c,Lost_Explanation__c,CR_Opp_Opted_out_Affiliate__c,CR_Opp_Restrictions_Affiliate__c,CR_Opp_Type_of_Processing_Fee__c,current_customer,MRR_local,currency_code,exchange_rate,mrr_usd,total_gmv_usd,cancelled_gmv_usd,succesful_gmv_usd,processing_fee_pct,succesful_gmv_local,processing_income_local,processing_income_usd,name_facility,store_gmv_mrr,gmv_mrr_cat,succesful_orders,total_cu_ft,pct_scanned,CSM__c,CR_Account_Categories__c,CR_Account_Sub_category__c,CR_Account_Cuisine_Type__c,CR_Acc_Social_Media_Presence__c,LAM_TierScrape__c,CR_Type_of_Customer__c,orders_10picked,CR_Acc_Social_Media_Presence_Numeric__c
0,2023-05-01,Brazil,e445cbbd-b698-43cf-9ccc-4362d07b4381,f7a65e9f-3342-48d2-a0fb-e9e4633c4bf6,RIO - Tijuca,False,435,13,0,13,424,17267.54,470.99,2,1,1,1.974713,4.803571,None,NaN,Brownice - Tijuca - Doces e Sobremesas - Pik n...,NaN,2023-08-22 14:59:32.775832-05:00,0.029885,0.000000,0.029885,Brownice,0065G00000j7FmKQAU,CR - Baked Goods,Rio de Janeiro,Brazil,False,NaT,5.0,930.0,0.0,1395.0,2021-12-28,2022-05-02,2022-05-02,NaN,None,None,False,False,None,False,2325.0,BRL,5.01,464.071856,3446.614770,94.009980,3352.604790,0.05,16796.55,839.8275,167.63024,Brownice - Tijuca - Doces e Sobremesas - Pik n...,7.224323,5x to 9.9x,422,20.0,0.974713,0056T000009cfwEQAQ,CR - Baked Goods,CR - Brownies,CR - Baked Goods,10k-20k,Tier 2,None,395,12000.0
1,2022-09-01,Mexico,45474a0a-ca40-42ed-9ab2-1ad008a834b5,b9545910-bca6-4b2a-a603-363721e7a23d,MEX - Santa Fe,True,211,6,1,7,204,49277.76,401.55,2,1,1,2.203791,4.466667,0:02:52.720,2.508475,Stuffed,2.866667,2023-08-22 14:59:32.775832-05:00,0.028436,0.004739,0.033175,Stuffed,0065G00000nahZ7QAI,CR - Baked Goods,Mexico City,Mexico,True,2022-12-31,10.0,1500.0,0.0,1000.0,2022-02-24,2022-03-24,2022-03-24,NaN,None,None,False,False,None,False,2500.0,MXN,20.06,124.626122,2456.518445,20.017448,2436.500997,0.10,48876.21,4887.6210,243.65010,Stuffed-MEX - Santa Fe,19.550484,15x or more,205,9.0,0.966825,0056T000009MdzFQAS,CR - Baked Goods,CR - Cookies,CR - Baked Goods,5k-7k,Tier 1,None,194,5736.0


In [78]:
# Renaming to new convention
combined_data_fx_2 = combined_data_fx.rename(columns={'created_month' : "A_Created_Month",
                                                     'country' : "Country",
                                                     'org_id' : "Organization_Id",
                                                     'facility_id' : "Facility_Id",
                                                     'facility_name' : "Facility_Name",
                                                     'org_deleted_skusku' : "Org_Deleted_SKU",
                                                     'total_orders' : "Orders_All",
                                                     'cancelled_orders' : "Orders_Cancelled",
                                                     'issue_orders' : "Orders_Issue",
                                                     'ODR_orders' : "Orders_ODR",
                                                     'scanned_orders' : "Orders_Scanned",
                                                     'total_gmv_local' : "GMV_Loc_All",
                                                     'cancelled_gmv_local' : "GMV_Loc_Cancelled",
                                                     'OFOs' : "Active_OFOs",
                                                     'cities' : "Active_Cities",
                                                     'facilities' : "Active_Facilities",
                                                     'Avg_Items_per_Order' : "Avg_SKUs_Order",
                                                     'avg_rating' : "Avg_Rating",
                                                     'avg_picking_time' : "Avg_PickingTimestamp",
                                                     'avg_picking_time_minutes' : "Avg_PickingMinutes",
                                                     'name' : "Account_Name",
                                                     'avg_picking_mins' : "Avg_Picking_Mins_Calc",
                                                     'data_last_updated' : "Data_Last_Updated",
                                                     'pct_cancelled' : "Orders_Perc_Cancelled",
                                                     'pct_issues' : "Orders_Perc_Issues",
                                                     'pct_odr' : "Orders_Perc_ODR",
                                                     'acct_name_jira' : "SF_Account_Name",
                                                     'Opportunity_ID_18__c' : "SF_Opportunity_Id",
                                                     'Cuisine_Type__c' : "SF_Cuisine_Type",
                                                     'kitchen_city' : "SF_City",
                                                     'kitchen_country' : "SF_Country",
                                                     'is_churned' : "SF_Is_Churned",
                                                     'churn_date' : "SF_Churn_Date",
                                                     'processing_fee_int' : "SF_Processing_Fee_int",
                                                     'Monthly_Shelf_Fee_Dry_Storage__c' : "SF_Dry_Storage_Fee",
                                                     'Monthly_Shelf_Fee_Frozen_Storage__c' : "SF_Frozen_Storage_Fee",
                                                     'Monthly_Shelf_Fee_Cold_Storage__c' : "SF_Cold_Storage_Fee",
                                                     'closed_won_date' : "SF_Closed_Won_Date",
                                                     'Move_In_Date__c': "SF_Move_In_Date",
                                                     'Start_Date__c' : "SF_Start_Date",
                                                     'Loss_Reason__c' : "SF_Loss_Reason",
                                                     'Specific_Lost_Reason__c' : "SF_Specific_Loss_Reason",
                                                     'Lost_Explanation__c' : "SF_Loss_Explanation",
                                                     'CR_Opp_Opted_out_Affiliate__c': "SF_Opp_Opted_Out_Affiliate",
                                                     'CR_Opp_Restrictions_Affiliate__c' : "SF_Opp_Restrictions_Affiliate",
                                                     'CR_Opp_Type_of_Processing_Fee__c' : "SF_Opp_Type_of_Processing_Fee",
                                                     'current_customer' : "SF_Current_Customer",
                                                     #'Org_Id_Otter__c',  
                                                     'MRR_local' : "SF_MRR_Loc",
                                                     'currency_code' : "Finance_Currency",
                                                     'exchange_rate' : "Finance_FX",
                                                     'mrr_usd' : "SF_MRR_USD",
                                                     'total_gmv_usd' : "GMV_USD_All",
                                                     'cancelled_gmv_usd' : "GMV_USD_Cancelled",
                                                     'succesful_gmv_usd' : "GMV_USD_Successful",
                                                     'processing_fee_pct' : "SF_Processing_Fee_Perc",
                                                     'succesful_gmv_local' : "GMV_Loc_Successful",
                                                     'processing_income_local' : "Income_PF_Loc",
                                                     'processing_income_usd' : "Income_PF_USD",
                                                     'name_facility' : "Concat_AccName_Facility",
                                                     'store_gmv_mrr' : "GMV_vs_MRR",
                                                     'gmv_mrr_cat' : "GMV_MRR_Group",
                                                     'succesful_orders' : "Orders_Successful",
                                                     'total_cu_ft' : "SF_Storage_Cuft",
                                                     'pct_scanned' : "Orders_Perc_Scanned",
                                                     'CSM__c' : "SF_CSM",
                                                     'CR_Account_Categories__c' : "SF_Category",
                                                     'CR_Account_Sub_category__c' : "SF_Subcategory",
                                                     'CR_Account_Cuisine_Type__c' : "SF_Account_Cuisine_Type",
                                                     'CR_Acc_Social_Media_Presence__c' : "SF_SocialMediaPresence",
                                                     'LAM_TierScrape__c' : "SF_TierScrape",
                                                     'CR_Type_of_Customer__c' : "SF_TypeofCustomer"
                                                     ,'orders_10picked':'orders_10picked_SLA_orders'
                                                     ,'CR_Acc_Social_Media_Presence_Numeric__c':'SF_SocialMediaPresence_Numeric'})

In [79]:
# Inspect
combined_data_fx_2.head(2)

,A_Created_Month,Country,Organization_Id,Facility_Id,Facility_Name,Org_Deleted_SKU,Orders_All,Orders_Cancelled,Orders_Issue,Orders_ODR,Orders_Scanned,GMV_Loc_All,GMV_Loc_Cancelled,Active_OFOs,Active_Cities,Active_Facilities,Avg_SKUs_Order,Avg_Rating,Avg_PickingTimestamp,Avg_PickingMinutes,Account_Name,Avg_Picking_Mins_Calc,Data_Last_Updated,Orders_Perc_Cancelled,Orders_Perc_Issues,Orders_Perc_ODR,SF_Account_Name,SF_Opportunity_Id,SF_Cuisine_Type,SF_City,SF_Country,SF_Is_Churned,SF_Churn_Date,SF_Processing_Fee_int,SF_Dry_Storage_Fee,SF_Frozen_Storage_Fee,SF_Cold_Storage_Fee,SF_Closed_Won_Date,SF_Move_In_Date,SF_Start_Date,SF_Loss_Reason,SF_Specific_Loss_Reason,SF_Loss_Explanation,SF_Opp_Opted_Out_Affiliate,SF_Opp_Restrictions_Affiliate,SF_Opp_Type_of_Processing_Fee,SF_Current_Customer,SF_MRR_Loc,Finance_Currency,Finance_FX,SF_MRR_USD,GMV_USD_All,GMV_USD_Cancelled,GMV_USD_Successful,SF_Processing_Fee_Perc,GMV_Loc_Successful,Income_PF_Loc,Income_PF_USD,Concat_AccName_Facility,GMV_vs_MRR,GMV_MRR_Group,Orders_Successful,SF_Storage_Cuft,Orders_Perc_Scanned,SF_CSM,SF_Category,SF_Subcategory,SF_Account_Cuisine_Type,SF_SocialMediaPresence,SF_TierScrape,SF_TypeofCustomer,orders_10picked_SLA_orders,SF_SocialMediaPresence_Numeric
0,2023-05-01,Brazil,e445cbbd-b698-43cf-9ccc-4362d07b4381,f7a65e9f-3342-48d2-a0fb-e9e4633c4bf6,RIO - Tijuca,False,435,13,0,13,424,17267.54,470.99,2,1,1,1.974713,4.803571,None,NaN,Brownice - Tijuca - Doces e Sobremesas - Pik n...,NaN,2023-08-22 14:59:32.775832-05:00,0.029885,0.000000,0.029885,Brownice,0065G00000j7FmKQAU,CR - Baked Goods,Rio de Janeiro,Brazil,False,NaT,5.0,930.0,0.0,1395.0,2021-12-28,2022-05-02,2022-05-02,NaN,None,None,False,False,None,False,2325.0,BRL,5.01,464.071856,3446.614770,94.009980,3352.604790,0.05,16796.55,839.8275,167.63024,Brownice - Tijuca - Doces e Sobremesas - Pik n...,7.224323,5x to 9.9x,422,20.0,0.974713,0056T000009cfwEQAQ,CR - Baked Goods,CR - Brownies,CR - Baked Goods,10k-20k,Tier 2,None,395,12000.0
1,2022-09-01,Mexico,45474a0a-ca40-42ed-9ab2-1ad008a834b5,b9545910-bca6-4b2a-a603-363721e7a23d,MEX - Santa Fe,True,211,6,1,7,204,49277.76,401.55,2,1,1,2.203791,4.466667,0:02:52.720,2.508475,Stuffed,2.866667,2023-08-22 14:59:32.775832-05:00,0.028436,0.004739,0.033175,Stuffed,0065G00000nahZ7QAI,CR - Baked Goods,Mexico City,Mexico,True,2022-12-31,10.0,1500.0,0.0,1000.0,2022-02-24,2022-03-24,2022-03-24,NaN,None,None,False,False,None,False,2500.0,MXN,20.06,124.626122,2456.518445,20.017448,2436.500997,0.10,48876.21,4887.6210,243.65010,Stuffed-MEX - Santa Fe,19.550484,15x or more,205,9.0,0.966825,0056T000009MdzFQAS,CR - Baked Goods,CR - Cookies,CR - Baked Goods,5k-7k,Tier 1,None,194,5736.0


**MRR / GMV report uploading**

In [80]:
# Uploading to Google Sheets
print("Authenticating and opening spreadsheet...")
gc = pygsheets.authorize(custom_credentials=credentials)
print("Opening Spreadsheet..")
ss = gc.open_by_key("1nXLC0Ccu_hznyC8Ylci2BXlXnfwdv7GJFIVBc61I40U")

# Selecting location
ws = ss.worksheet("title","raw_data")

# Uploading data
print("Clearing sheet ...")
ws.clear(start="A", end="BU")
print("Sheet cleared")

print("Uploading data...")
ws.set_dataframe(combined_data_fx_2, start= "A1",copy_head= True, extend= True, nan=  "")
print("Data uploaded!")

Authenticating and opening spreadsheet...
Opening Spreadsheet..
Clearing sheet ...
Sheet cleared
Uploading data...
Data uploaded!


**GMV Report to LATAM MBR Spreadsheet**

In [81]:
# Uploading to Google Sheets (LATAM MBR report)
print("Authenticating and opening spreadsheet...")
gc = pygsheets.authorize(custom_credentials=credentials)
print("Opening Spreadsheet..")
ss = gc.open_by_key("1DikVCmJQe6y02vUgIEaUE-FoPmzj7erxUXCU_jkX63M")

# Selecting location
ws = ss.worksheet("title","gmv_mrr_opp")

# Uploading data
print("Clearing sheet ...")
ws.clear(start="A", end="BU")
print("Sheet cleared")

print("Uploading data...")
ws.set_dataframe(combined_data_fx_2, start= "A1",copy_head= True, extend= True, nan=  "")
print("Data uploaded!")

Authenticating and opening spreadsheet...
Opening Spreadsheet..
Clearing sheet ...
Sheet cleared
Uploading data...
Data uploaded!


In [82]:
# Uploading to Google Sheets (CR Time to healthy)
print("Authenticating and opening spreadsheet...")
gc = pygsheets.authorize(custom_credentials=credentials)
print("Opening Spreadsheet..")
ss = gc.open_by_key("1kgEK_xf-K7XpXjYtq-OshRR711aT50iT4e-Pfo0ayEE")

# Selecting location
ws = ss.worksheet("title","D_LATAM_DOORS")

# Uploading data
print("Clearing sheet ...")
ws.clear(start="A", end="BU")
print("Sheet cleared")

print("Uploading data...")
ws.set_dataframe(combined_data_fx_2, start= "A1",copy_head= True, extend= True, nan=  "")
print("Data uploaded!")

Authenticating and opening spreadsheet...
Opening Spreadsheet..
Clearing sheet ...
Sheet cleared
Uploading data...
Data uploaded!


In [83]:
# Uploading to Google Sheets (CR LATAM perfect launch)
print("Authenticating and opening spreadsheet...")
gc = pygsheets.authorize(custom_credentials=credentials)
print("Opening Spreadsheet..")
ss = gc.open_by_key("1JyuBtS0XtAm3KGCCnO4IJHfIciVilp0FRSLLwbP9OPo")

# Selecting location
ws = ss.worksheet("title","D_gmv_mrr_opp")

# Uploading data
print("Clearing sheet ...")
ws.clear(start="A", end="BU")
print("Sheet cleared")

print("Uploading data...")
ws.set_dataframe(combined_data_fx_2, start= "A1",copy_head= True, extend= True, nan=  "")
print("Data uploaded!")

Authenticating and opening spreadsheet...
Opening Spreadsheet..
Clearing sheet ...
Sheet cleared
Uploading data...
Data uploaded!


**Report to GBQ**

In [84]:
# Import libraries for GBQ
from pandas.io import gbq
from google.oauth2 import service_account
from google.cloud import bigquery

In [85]:
combined_data_fx.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6936 entries, 0 to 6935
Data columns (total 73 columns):
 #   Column                                   Non-Null Count  Dtype                         
---  ------                                   --------------  -----                         
 0   created_month                            6936 non-null   datetime64[ns]                
 1   country                                  6936 non-null   object                        
 2   org_id                                   6936 non-null   object                        
 3   facility_id                              6936 non-null   object                        
 4   facility_name                            6936 non-null   object                        
 5   org_deleted_skusku                       6936 non-null   bool                          
 6   total_orders                             6936 non-null   int64                         
 7   cancelled_orders                         6936 non-n

In [86]:
# Upload data to GBQ (original DB without the new names and columns nov 22)

print("Uploading data to GBQ...")

table_id = 'css-operations.latam_cr_bizops.gmv_mrr_opp'
project_id = 'css-operations'
credentials = service_account.Credentials.from_service_account_info(s.get_secret("cloud_retail_sp_service_account"), scopes = SCOPES)
pandas_gbq.to_gbq(combined_data_fx,table_id, project_id = project_id, if_exists = 'replace', credentials = credentials)

print("Data uploaded!")

Uploading data to GBQ...


1it [00:06,  6.18s/it]

Data uploaded!


**Upload to Sofis Op plan**

- filter data from jan 2023 onwards

In [87]:
combined_data_fx.head(2)

,created_month,country,org_id,facility_id,facility_name,org_deleted_skusku,total_orders,cancelled_orders,issue_orders,ODR_orders,scanned_orders,total_gmv_local,cancelled_gmv_local,OFOs,cities,facilities,Avg_Items_per_Order,avg_rating,avg_picking_time,avg_picking_time_minutes,name,avg_picking_mins,data_last_updated,pct_cancelled,pct_issues,pct_odr,acct_name_jira,Opportunity_ID_18__c,Cuisine_Type__c,kitchen_city,kitchen_country,is_churned,churn_date,processing_fee_int,Monthly_Shelf_Fee_Dry_Storage__c,Monthly_Shelf_Fee_Frozen_Storage__c,Monthly_Shelf_Fee_Cold_Storage__c,closed_won_date,Move_In_Date__c,Start_Date__c,Loss_Reason__c,Specific_Lost_Reason__c,Lost_Explanation__c,CR_Opp_Opted_out_Affiliate__c,CR_Opp_Restrictions_Affiliate__c,CR_Opp_Type_of_Processing_Fee__c,current_customer,MRR_local,currency_code,exchange_rate,mrr_usd,total_gmv_usd,cancelled_gmv_usd,succesful_gmv_usd,processing_fee_pct,succesful_gmv_local,processing_income_local,processing_income_usd,name_facility,store_gmv_mrr,gmv_mrr_cat,succesful_orders,total_cu_ft,pct_scanned,CSM__c,CR_Account_Categories__c,CR_Account_Sub_category__c,CR_Account_Cuisine_Type__c,CR_Acc_Social_Media_Presence__c,LAM_TierScrape__c,CR_Type_of_Customer__c,orders_10picked,CR_Acc_Social_Media_Presence_Numeric__c
0,2023-05-01,Brazil,e445cbbd-b698-43cf-9ccc-4362d07b4381,f7a65e9f-3342-48d2-a0fb-e9e4633c4bf6,RIO - Tijuca,False,435,13,0,13,424,17267.54,470.99,2,1,1,1.974713,4.803571,None,NaN,Brownice - Tijuca - Doces e Sobremesas - Pik n...,NaN,2023-08-22 14:59:32.775832-05:00,0.029885,0.000000,0.029885,Brownice,0065G00000j7FmKQAU,CR - Baked Goods,Rio de Janeiro,Brazil,False,NaT,5.0,930.0,0.0,1395.0,2021-12-28,2022-05-02,2022-05-02,NaN,None,None,False,False,None,False,2325.0,BRL,5.01,464.071856,3446.614770,94.009980,3352.604790,0.05,16796.55,839.8275,167.63024,Brownice - Tijuca - Doces e Sobremesas - Pik n...,7.224323,5x to 9.9x,422,20.0,0.974713,0056T000009cfwEQAQ,CR - Baked Goods,CR - Brownies,CR - Baked Goods,10k-20k,Tier 2,None,395,12000.0
1,2022-09-01,Mexico,45474a0a-ca40-42ed-9ab2-1ad008a834b5,b9545910-bca6-4b2a-a603-363721e7a23d,MEX - Santa Fe,True,211,6,1,7,204,49277.76,401.55,2,1,1,2.203791,4.466667,0:02:52.720,2.508475,Stuffed,2.866667,2023-08-22 14:59:32.775832-05:00,0.028436,0.004739,0.033175,Stuffed,0065G00000nahZ7QAI,CR - Baked Goods,Mexico City,Mexico,True,2022-12-31,10.0,1500.0,0.0,1000.0,2022-02-24,2022-03-24,2022-03-24,NaN,None,None,False,False,None,False,2500.0,MXN,20.06,124.626122,2456.518445,20.017448,2436.500997,0.10,48876.21,4887.6210,243.65010,Stuffed-MEX - Santa Fe,19.550484,15x or more,205,9.0,0.966825,0056T000009MdzFQAS,CR - Baked Goods,CR - Cookies,CR - Baked Goods,5k-7k,Tier 1,None,194,5736.0


In [88]:
# Filter dates
gmv_mrr_jan = combined_data_fx[combined_data_fx["created_month"]>= '2023-01-01']
gmv_mrr_jan.head()

,created_month,country,org_id,facility_id,facility_name,org_deleted_skusku,total_orders,cancelled_orders,issue_orders,ODR_orders,scanned_orders,total_gmv_local,cancelled_gmv_local,OFOs,cities,facilities,Avg_Items_per_Order,avg_rating,avg_picking_time,avg_picking_time_minutes,name,avg_picking_mins,data_last_updated,pct_cancelled,pct_issues,pct_odr,acct_name_jira,Opportunity_ID_18__c,Cuisine_Type__c,kitchen_city,kitchen_country,is_churned,churn_date,processing_fee_int,Monthly_Shelf_Fee_Dry_Storage__c,Monthly_Shelf_Fee_Frozen_Storage__c,Monthly_Shelf_Fee_Cold_Storage__c,closed_won_date,Move_In_Date__c,Start_Date__c,Loss_Reason__c,Specific_Lost_Reason__c,Lost_Explanation__c,CR_Opp_Opted_out_Affiliate__c,CR_Opp_Restrictions_Affiliate__c,CR_Opp_Type_of_Processing_Fee__c,current_customer,MRR_local,currency_code,exchange_rate,mrr_usd,total_gmv_usd,cancelled_gmv_usd,succesful_gmv_usd,processing_fee_pct,succesful_gmv_local,processing_income_local,processing_income_usd,name_facility,store_gmv_mrr,gmv_mrr_cat,succesful_orders,total_cu_ft,pct_scanned,CSM__c,CR_Account_Categories__c,CR_Account_Sub_category__c,CR_Account_Cuisine_Type__c,CR_Acc_Social_Media_Presence__c,LAM_TierScrape__c,CR_Type_of_Customer__c,orders_10picked,CR_Acc_Social_Media_Presence_Numeric__c
0,2023-05-01,Brazil,e445cbbd-b698-43cf-9ccc-4362d07b4381,f7a65e9f-3342-48d2-a0fb-e9e4633c4bf6,RIO - Tijuca,False,435,13,0,13,424,17267.54,470.99,2,1,1,1.974713,4.803571,None,NaN,Brownice - Tijuca - Doces e Sobremesas - Pik n...,NaN,2023-08-22 14:59:32.775832-05:00,0.029885,0.0,0.029885,Brownice,0065G00000j7FmKQAU,CR - Baked Goods,Rio de Janeiro,Brazil,False,NaT,5.0,930.0,0.0,1395.0,2021-12-28,2022-05-02,2022-05-02,NaN,None,None,False,False,None,False,2325.0,BRL,5.01,464.071856,3446.614770,94.009980,3352.604790,0.05,16796.55,839.8275,167.630240,Brownice - Tijuca - Doces e Sobremesas - Pik n...,7.224323,5x to 9.9x,422,20.0,0.974713,0056T000009cfwEQAQ,CR - Baked Goods,CR - Brownies,CR - Baked Goods,10k-20k,Tier 2,None,395,12000.0
4,2023-07-01,Brazil,ac0f7e69-d5fa-49c9-8c70-b3fb7523de43,4518d99c-fa47-4da7-b5f5-c071c78b5ac2,BHZ - Tancredo Neves,False,190,2,0,2,182,7508.76,139.87,1,1,1,2.178947,4.631579,None,NaN,Carolline Doceria - Pik n' Pak,NaN,2023-08-22 14:59:32.775832-05:00,0.010526,0.0,0.010526,Carolline doceria,0066T000019GxH6QAK,CR - Baked Goods,Belo Horizonte,Brazil,False,NaT,10.0,0.0,0.0,318.0,2022-12-30,2023-01-18,2023-01-30,NaN,None,None,False,False,None,False,318.0,BRL,4.80,66.250000,1564.325000,29.139583,1535.185417,0.10,7368.89,736.8890,153.518542,Carolline Doceria - Pik n' Pak-BHZ - Tancredo...,23.172610,15x or more,188,4.0,0.957895,005f40000054nEXAAY,CR - Baked Goods,None,CR - Baked Goods,10k-20k,Tier 3,Local,154,14000.0
5,2023-03-01,Brazil,ee01b7c0-d553-4673-bd31-b97d7e8036d3,f7a65e9f-3342-48d2-a0fb-e9e4633c4bf6,RIO - Tijuca,False,69,2,0,2,67,2663.44,60.98,1,1,1,1.347826,4.857143,None,NaN,Hummm... Pudim! - Pik n' Pak,NaN,2023-08-22 14:59:32.775832-05:00,0.028986,0.0,0.028986,Hummm... Pudim!,0066T000018ak4fQAA,CR - Baked Goods,Rio de Janeiro,Brazil,False,NaT,10.0,0.0,0.0,400.0,2022-08-11,2023-02-28,2022-12-19,NaN,None,None,False,False,None,False,400.0,BRL,5.22,76.628352,510.237548,11.681992,498.555556,0.10,2602.46,260.2460,49.855556,Hummm... Pudim! - Pik n' Pak-RIO - Tijuca,6.506150,5x to 9.9x,67,4.0,0.971014,005f40000054nEXAAY,CR - Desserts,CR - Flan,CR - Baked Goods,1k-3k,Tier 3,Local,63,1385.0
8,2023-07-01,Mexico,e53bbc96-9c1e-47e0-9f66-df868664c75f,b9545910-bca6-4b2a-a603-363721e7a23d,MEX - Santa Fe,False,189,5,0,5,186,49418.00,896.00,3,1,1,5.873016,4.913043,None,NaN,La Pantera Fresca - CR,NaN,2023-08-22 14:59:32.775832-05:00,0.026455,0.0,0.026455,La pantera fresca,006f400000Vx1XQAAZ,"CR - Ice Cream, Popsicle, Cookie Dough & Gelato",Mexico City,Mexico,False,NaT,5.0,0.0,5000.0,0.0,2020-08-31,2020-10-08,2020-10-01,NaN,None,None,False,False,None,True,5000.0,MXN,16.93,295.333727,2918.960425,52.923804,2866.036621,0.05,48522.00,2426.1000,143.301831,La Pantera Fresca - CR-ME

In [89]:
gmv_mrr_jan["created_month"].unique()

array(['2023-05-01T00:00:00.000000000', '2023-07-01T00:00:00.000000000',
       '2023-03-01T00:00:00.000000000', '2023-06-01T00:00:00.000000000',
       '2023-04-01T00:00:00.000000000', '2023-08-01T00:00:00.000000000',
       '2023-01-01T00:00:00.000000000', '2023-02-01T00:00:00.000000000'],
      dtype='datetime64[ns]')

In [90]:
# Upload to sheet
print("Authenticating and opening spreadsheet...")
gc = pygsheets.authorize(custom_credentials=credentials)
print("Opening Spreadsheet..")
ss = gc.open_by_key("1o_predpW4A-Xj0Z3LCbU_S1sALR7bOWeHvuO-d4r2vI")

# Selecting location
ws = ss.worksheet("title","D_gmv_mrr_opp_latam")

# Uploading data
print("Clearing sheet ...")
ws.clear(start="A", end="BU")
print("Sheet cleared")

print("Uploading data...")
ws.set_dataframe(gmv_mrr_jan, start= "A1",copy_head= True, extend= True, nan=  "")
print("Data uploaded!")

Authenticating and opening spreadsheet...
Opening Spreadsheet..
Clearing sheet ...
Sheet cleared
Uploading data...
Data uploaded!


**Churn Report**


In [91]:
sf_data.sample(2)

,attributes,Account,Account_Name_Jira__c,Opportunity_ID_18__c,Name,Facility_Name__c,Cuisine_Type__c,Id,Kitchen_City__c,Kitchen_Country__c,Is_Churned__c,Churn_Date__c,Processing_Fee__c,Monthly_Shelf_Fee_Dry_Storage__c,Monthly_Shelf_Fee_Frozen_Storage__c,Monthly_Shelf_Fee_Cold_Storage__c,Closed_Won_Date__c,Move_In_Date__c,Start_Date__c,Loss_Reason__c,Specific_Lost_Reason__c,Lost_Explanation__c,CR_Opp_Opted_out_Affiliate__c,CR_Opp_Restrictions_Affiliate__c,CR_Opp_Type_of_Processing_Fee__c,Storage_Amount_Dry_CU_FT__c,Storage_Amount_Frozen_CU_FT__c,Storage_Amount_Cold_CU_FT__c,sfdc_org_id,current_customer,org_id,Org_Id_Otter__c,CSM__c,CR_Account_Categories__c,CR_Account_Sub_category__c,CR_Account_Cuisine_Type__c,CR_Acc_Social_Media_Presence__c,LAM_TierScrape__c,CR_Type_of_Customer__c,CR_Acc_Social_Media_Presence_Numeric__c
2038,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",Los Paleteros,0065G00000aEfhKQAS,Los Paleteros - (franqueador) - SAO - Batista ...,SAO - Batista,"CR - Ice Cream, Popsicle, Cookie Dough & Gelato",0065G00000aEfhKQAS,Sao Paulo,Brazil,True,2022-07-11,9.0,0.0,1350.0,0.0,2021-07-16,None,2022-07-01,None,None,"Stale Opportunities Jul 09, 2021",False,False,None,0.0,20.0,0.0,be169c79-231d-48cd-81b1-23800c573374,True,be169c79-231d-48cd-81b1-23800c573374,be169c79-231d-48cd-81b1-23800c573374,0055G0000080AXnQAM,CR - Frozen Desserts,CR - Popsicle,"CR - Ice Cream, Popsicle, Cookie Dough & Gelato",1k-3k,Tier 2,None,1930.0
510,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",Doceria Maktub,0066T000019HQVTQA4,Doceria Maktub - RIO - Barra,RIO - Barra,CR - Baked Goods,0066T000019HQVTQA4,Rio de Janeiro,Brazil,True,2023-05-31,10.0,600.0,0.0,600.0,2022-11-28,2023-01-30,2023-01-30,None,None,None,False,False,None,8.0,0.0,8.0,f1eb8923-ef20-494c-bbc9-322c8ebf91bc,False,f1eb8923-ef20-494c-bbc9-322c8ebf91bc,f1eb8923-ef20-494c-bbc9-322c8ebf91bc,None,CR - Baked Goods,None,CR - Baked Goods,5k-7k,Tier 3,Local,5844.0


In [92]:
churn_sf = sf_data.copy()

In [93]:
# Convert to datetime
churn_sf["Closed_Won_Date__c"] = pd.to_datetime(churn_sf["Closed_Won_Date__c"])

# Truncate date to month
churn_sf["closed_won_date"] = churn_sf['Closed_Won_Date__c'].dt.to_period('M').dt.to_timestamp()

In [94]:
latam_monthly.head(2)

,created_month,country,currency_code,exchange_rate
0,2019-01-01,Mexico,MXN,18.80
1,2019-02-01,Mexico,MXN,19.59


In [95]:
latam_monthly["created_month"] = pd.to_datetime(latam_monthly["created_month"])

In [96]:
churn_sf_fx = churn_sf.merge(latam_monthly, how = "left", left_on = ["closed_won_date", "Kitchen_Country__c"], right_on = ["created_month", "country"])

In [97]:
# Create total local mrr column
churn_sf_fx["mrr_local"] = churn_sf_fx["Monthly_Shelf_Fee_Dry_Storage__c"] + churn_sf_fx["Monthly_Shelf_Fee_Frozen_Storage__c"] + churn_sf_fx["Monthly_Shelf_Fee_Cold_Storage__c"]

# Create mrr in usd
churn_sf_fx["mrr_usd"] = churn_sf_fx["mrr_local"] / churn_sf_fx["exchange_rate"]

In [98]:
# Change created_month to datetime
combined_data_fx["created_month"] = pd.to_datetime(combined_data_fx["created_month"])

In [99]:
# Import credentials
path = "/home/shared/joseph.jensen/Alan Data Pipelines/Credentials/sf_auth_JJ.json"

loginInfo = json.load(open(path))
session_id, instance = SalesforceLogin(username= loginInfo["username"],
                                                 password= loginInfo["password"],
                                                 security_token= loginInfo["security_token"],
                                                 domain= "login")

# Set SF connection
sf = Salesforce(instance= instance, session_id= session_id)

In [100]:
additional_sf = """

SELECT
Account.Org_Id_Otter__c,
Churn_Type__c,
Churn_Reason__c,
Failure_to_Pay__c,
Secondary_Churn_Reasons_If_Applicable__c,
id

FROM
Opportunity

WHERE
RecordType.Name in ('Virtual', 'CloudRetail','virtual;cloudretail') AND (
Opp_Owner_Region__c in ('LATAM') OR Kitchen_Country__c IN ('Mexico', 'Colombia', 'Brazil'))
AND StageName = 'Closed Won' ORDER BY Closed_Won_Date__c DESC

"""


print("Querying data...")

# Query users to dataframe
additional_fields = pd.DataFrame(sf.query_all(additional_sf)['records'])
print("Data pulled!")

# Inspect 
additional_fields.head()
print("Rows:", len(additional_fields))

Querying data...
Data pulled!
Rows: 2775


In [101]:
# Inspect 
additional_fields.head()

,attributes,Account,Churn_Type__c,Churn_Reason__c,Failure_to_Pay__c,Secondary_Churn_Reasons_If_Applicable__c,Id
0,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",None,None,None,None,0066T00001BfHs7QAF
1,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",None,None,None,None,0066T00001C0a16QAB
2,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",None,None,None,None,0066T00001D6dObQAJ
3,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",None,None,None,None,0066T00001C0Zo2QAF
4,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",None,None,None,None,0066T00001C0ZxiQAF


In [102]:
# Extract account details from accounts list in accounts column
# Extract account details from accounts list in accounts column
org_ids_2 = [] # lista orgs id

# has_churned = [] # lista has churned
for i in additional_fields['Account']:
    try:
        j=(i.get('Org_Id_Otter__c'))
        #print(j)
        org_ids_2.append(j)
    except:
        org_ids_2.append(None)

In [103]:
# Append extracted lists to DF
additional_fields['sfdc_org_id']= org_ids_2

In [104]:
# Fill NAs with bfill 
additional_fields['org_id']= additional_fields[["sfdc_org_id"]].bfill(axis=1).iloc[:, 0]

In [105]:
additional_fields.head()

,attributes,Account,Churn_Type__c,Churn_Reason__c,Failure_to_Pay__c,Secondary_Churn_Reasons_If_Applicable__c,Id,sfdc_org_id,org_id
0,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",None,None,None,None,0066T00001BfHs7QAF,b2d4fbf2-5ddc-46ad-8207-36b7bf77c88a,b2d4fbf2-5ddc-46ad-8207-36b7bf77c88a
1,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",None,None,None,None,0066T00001C0a16QAB,a01f793e-ab3c-4de8-b99e-500be6d2977e,a01f793e-ab3c-4de8-b99e-500be6d2977e
2,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",None,None,None,None,0066T00001D6dObQAJ,7d2329f1-fd48-49b5-8b1f-0400316f1ffd,7d2329f1-fd48-49b5-8b1f-0400316f1ffd
3,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",None,None,None,None,0066T00001C0Zo2QAF,53f797b7-d31f-493f-a41f-c32ec09a0b92,53f797b7-d31f-493f-a41f-c32ec09a0b92
4,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",None,None,None,None,0066T00001C0ZxiQAF,c17f5cf9-88a9-445e-b777-58a88bf9efca,c17f5cf9-88a9-445e-b777-58a88bf9efca


In [106]:
# Select additional fields
add_cols = ["Id","Churn_Type__c","Churn_Reason__c","Failure_to_Pay__c","Secondary_Churn_Reasons_If_Applicable__c"]
additional_fields_2 = additional_fields[add_cols]

In [107]:
print(len(additional_fields_2))
additional_fields_2.head()

2775


,Id,Churn_Type__c,Churn_Reason__c,Failure_to_Pay__c,Secondary_Churn_Reasons_If_Applicable__c
0,0066T00001BfHs7QAF,None,None,None,None
1,0066T00001C0a16QAB,None,None,None,None
2,0066T00001D6dObQAJ,None,None,None,None
3,0066T00001C0Zo2QAF,None,None,None,None
4,0066T00001C0ZxiQAF,None,None,None,None


In [108]:
print(len(churn_sf_fx))
churn_sf_fx.head(2)

2487


,attributes,Account,Account_Name_Jira__c,Opportunity_ID_18__c,Name,Facility_Name__c,Cuisine_Type__c,Id,Kitchen_City__c,Kitchen_Country__c,Is_Churned__c,Churn_Date__c,Processing_Fee__c,Monthly_Shelf_Fee_Dry_Storage__c,Monthly_Shelf_Fee_Frozen_Storage__c,Monthly_Shelf_Fee_Cold_Storage__c,Closed_Won_Date__c,Move_In_Date__c,Start_Date__c,Loss_Reason__c,Specific_Lost_Reason__c,Lost_Explanation__c,CR_Opp_Opted_out_Affiliate__c,CR_Opp_Restrictions_Affiliate__c,CR_Opp_Type_of_Processing_Fee__c,Storage_Amount_Dry_CU_FT__c,Storage_Amount_Frozen_CU_FT__c,Storage_Amount_Cold_CU_FT__c,sfdc_org_id,current_customer,org_id,Org_Id_Otter__c,CSM__c,CR_Account_Categories__c,CR_Account_Sub_category__c,CR_Account_Cuisine_Type__c,CR_Acc_Social_Media_Presence__c,LAM_TierScrape__c,CR_Type_of_Customer__c,CR_Acc_Social_Media_Presence_Numeric__c,closed_won_date,created_month,country,currency_code,exchange_rate,mrr_local,mrr_usd
0,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",Shoppifit,0066T00001C0st4QAB,Shoppifit - MED - De Moda,MED - DModa,None,0066T00001C0st4QAB,Medellin,Colombia,False,None,0.0,0.0,0.0,0.0,2023-08-18,None,2023-09-18,None,None,None,True,False,Percentage / Order,8.0,0.0,0.0,None,False,None,None,None,None,None,None,<1k,None,None,NaN,2023-08-01,2023-08-01,Colombia,COP,4117.0,0.0,0.0
1,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",Shoppifit,0066T00001D7kTaQAJ,Shoppifit - BOG - San Felipe,BOG - San Felipe,None,0066T00001D7kTaQAJ,Bogota,Colombia,False,None,0.0,0.0,0.0,0.0,2023-08-18,None,2023-09-18,None,None,None,True,False,Percentage / Order,8.0,0.0,0.0,None,False,None,None,None,None,None,None,<1k,None,None,NaN,2023-08-01,2023-08-01,Colombia,COP,4117.0,0.0,0.0


In [109]:
print(len(churn_sf_fx))

2487


In [110]:
# Merge
churn_sf_fx_2 = churn_sf_fx.merge(additional_fields_2, how = "left", left_on = ["Opportunity_ID_18__c"], right_on = ["Id"])

In [111]:
print(len(churn_sf_fx_2))

2487


In [112]:
churn_sf_fx_2.head()

,attributes,Account,Account_Name_Jira__c,Opportunity_ID_18__c,Name,Facility_Name__c,Cuisine_Type__c,Id_x,Kitchen_City__c,Kitchen_Country__c,Is_Churned__c,Churn_Date__c,Processing_Fee__c,Monthly_Shelf_Fee_Dry_Storage__c,Monthly_Shelf_Fee_Frozen_Storage__c,Monthly_Shelf_Fee_Cold_Storage__c,Closed_Won_Date__c,Move_In_Date__c,Start_Date__c,Loss_Reason__c,Specific_Lost_Reason__c,Lost_Explanation__c,CR_Opp_Opted_out_Affiliate__c,CR_Opp_Restrictions_Affiliate__c,CR_Opp_Type_of_Processing_Fee__c,Storage_Amount_Dry_CU_FT__c,Storage_Amount_Frozen_CU_FT__c,Storage_Amount_Cold_CU_FT__c,sfdc_org_id,current_customer,org_id,Org_Id_Otter__c,CSM__c,CR_Account_Categories__c,CR_Account_Sub_category__c,CR_Account_Cuisine_Type__c,CR_Acc_Social_Media_Presence__c,LAM_TierScrape__c,CR_Type_of_Customer__c,CR_Acc_Social_Media_Presence_Numeric__c,closed_won_date,created_month,country,currency_code,exchange_rate,mrr_local,mrr_usd,Id_y,Churn_Type__c,Churn_Reason__c,Failure_to_Pay__c,Secondary_Churn_Reasons_If_Applicable__c
0,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",Shoppifit,0066T00001C0st4QAB,Shoppifit - MED - De Moda,MED - DModa,None,0066T00001C0st4QAB,Medellin,Colombia,False,None,0.0,0.0,0.0,0.0,2023-08-18,None,2023-09-18,None,None,None,True,False,Percentage / Order,8.0,0.0,0.0,None,False,None,None,None,None,None,None,<1k,None,None,NaN,2023-08-01,2023-08-01,Colombia,COP,4117.00,0.0,0.000000,0066T00001C0st4QAB,None,None,None,None
1,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",Shoppifit,0066T00001D7kTaQAJ,Shoppifit - BOG - San Felipe,BOG - San Felipe,None,0066T00001D7kTaQAJ,Bogota,Colombia,False,None,0.0,0.0,0.0,0.0,2023-08-18,None,2023-09-18,None,None,None,True,False,Percentage / Order,8.0,0.0,0.0,None,False,None,None,None,None,None,None,<1k,None,None,NaN,2023-08-01,2023-08-01,Colombia,COP,4117.00,0.0,0.000000,0066T00001D7kTaQAJ,None,None,None,None
2,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",Food to Save,0066T00001D7txlQAB,Food to Save - BHZ - Paracatu,BHZ - Paracatu,None,0066T00001D7txlQAB,Sao Paulo,Brazil,False,None,0.0,0.0,0.0,0.0,2023-08-18,2023-07-26,2023-08-24,None,None,None,True,False,None,48.0,0.0,0.0,f9373a87-ab71-4198-bc2c-93fa2a27bcf8,False,f9373a87-ab71-4198-bc2c-93fa2a27bcf8,f9373a87-ab71-4198-bc2c-93fa2a27bcf8,0055G0000080AXnQAM,CR - Packaged Food / Ready to Eat,CR - Marmitas,CR - Packaged Food / Ready to Eat,>100k,Tier 1,Local,135000.0,2023-08-01,2023-08-01,Brazil,BRL,4.94,0.0,0.000000,0066T00001D7txlQAB,None,None,None,None
3,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",Shoppifit,0066T00001D7kShQAJ,Shoppifit - BOG - Atabanza,BOG - Atabanza,None,0066T00001D7kShQAJ,Bogota,Colombia,False,None,0.0,0.0,0.0,0.0,2023-08-18,None,2023-09-18,None,None,None,True,False,Percentage / Order,8.0,0.0,0.0,None,False,None,None,None,None,None,None,<1k,None,None,NaN,2023-08-01,2023-08-01,Colombia,COP,4117.00,0.0,0.000000,0066T00001D7kShQAJ,None,None,None,None
4,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",Santa Rita ( helados y paletas ) - Empresa Activa,0066T00001D7ZWJQA3,Santa Rita ( helados y paletas ) - Empresa Act...,MEX - Polanco,None,0066T00001D7ZWJQA3,Mexico City,Mexico,False,None,8.0,0.0,1500.0,0.0,2023-08-17,None,2023-09-15,None,None,None,True,False,None,0.0,4.0,0.0,b368a673-374e-4990-b456-06fecf6ad9c9,False,b368a673-374e-4990-b456-06fecf6ad9c9,b368a673-374e-4990-b456-06fecf6ad9c9,None,CR - Frozen Desserts,CR - Ice Cream,None,10k-20k,None,Local,12000.0,2023-08-01,2023-08-01,Mexico,MXN,16.92,1500.0,88.652482,0066T00001D7ZWJQA3,None,None,None,None


In [113]:
# Drop duplicated Id column
churn_sf_fx_2.drop(columns = ["Id_x","Id_y","attributes","Account", "sfdc_org_id","Org_Id_Otter__c"], inplace = True)

In [114]:
churn_sf_fx_2.head()

,Account_Name_Jira__c,Opportunity_ID_18__c,Name,Facility_Name__c,Cuisine_Type__c,Kitchen_City__c,Kitchen_Country__c,Is_Churned__c,Churn_Date__c,Processing_Fee__c,Monthly_Shelf_Fee_Dry_Storage__c,Monthly_Shelf_Fee_Frozen_Storage__c,Monthly_Shelf_Fee_Cold_Storage__c,Closed_Won_Date__c,Move_In_Date__c,Start_Date__c,Loss_Reason__c,Specific_Lost_Reason__c,Lost_Explanation__c,CR_Opp_Opted_out_Affiliate__c,CR_Opp_Restrictions_Affiliate__c,CR_Opp_Type_of_Processing_Fee__c,Storage_Amount_Dry_CU_FT__c,Storage_Amount_Frozen_CU_FT__c,Storage_Amount_Cold_CU_FT__c,current_customer,org_id,CSM__c,CR_Account_Categories__c,CR_Account_Sub_category__c,CR_Account_Cuisine_Type__c,CR_Acc_Social_Media_Presence__c,LAM_TierScrape__c,CR_Type_of_Customer__c,CR_Acc_Social_Media_Presence_Numeric__c,closed_won_date,created_month,country,currency_code,exchange_rate,mrr_local,mrr_usd,Churn_Type__c,Churn_Reason__c,Failure_to_Pay__c,Secondary_Churn_Reasons_If_Applicable__c
0,Shoppifit,0066T00001C0st4QAB,Shoppifit - MED - De Moda,MED - DModa,None,Medellin,Colombia,False,None,0.0,0.0,0.0,0.0,2023-08-18,None,2023-09-18,None,None,None,True,False,Percentage / Order,8.0,0.0,0.0,False,None,None,None,None,None,<1k,None,None,NaN,2023-08-01,2023-08-01,Colombia,COP,4117.00,0.0,0.000000,None,None,None,None
1,Shoppifit,0066T00001D7kTaQAJ,Shoppifit - BOG - San Felipe,BOG - San Felipe,None,Bogota,Colombia,False,None,0.0,0.0,0.0,0.0,2023-08-18,None,2023-09-18,None,None,None,True,False,Percentage / Order,8.0,0.0,0.0,False,None,None,None,None,None,<1k,None,None,NaN,2023-08-01,2023-08-01,Colombia,COP,4117.00,0.0,0.000000,None,None,None,None
2,Food to Save,0066T00001D7txlQAB,Food to Save - BHZ - Paracatu,BHZ - Paracatu,None,Sao Paulo,Brazil,False,None,0.0,0.0,0.0,0.0,2023-08-18,2023-07-26,2023-08-24,None,None,None,True,False,None,48.0,0.0,0.0,False,f9373a87-ab71-4198-bc2c-93fa2a27bcf8,0055G0000080AXnQAM,CR - Packaged Food / Ready to Eat,CR - Marmitas,CR - Packaged Food / Ready to Eat,>100k,Tier 1,Local,135000.0,2023-08-01,2023-08-01,Brazil,BRL,4.94,0.0,0.000000,None,None,None,None
3,Shoppifit,0066T00001D7kShQAJ,Shoppifit - BOG - Atabanza,BOG - Atabanza,None,Bogota,Colombia,False,None,0.0,0.0,0.0,0.0,2023-08-18,None,2023-09-18,None,None,None,True,False,Percentage / Order,8.0,0.0,0.0,False,None,None,None,None,None,<1k,None,None,NaN,2023-08-01,2023-08-01,Colombia,COP,4117.00,0.0,0.000000,None,None,None,None
4,Santa Rita ( helados y paletas ) - Empresa Activa,0066T00001D7ZWJQA3,Santa Rita ( helados y paletas ) - Empresa Act...,MEX - Polanco,None,Mexico City,Mexico,False,None,8.0,0.0,1500.0,0.0,2023-08-17,None,2023-09-15,None,None,None,True,False,None,0.0,4.0,0.0,False,b368a673-374e-4990-b456-06fecf6ad9c9,None,CR - Frozen Desserts,CR - Ice Cream,None,10k-20k,None,Local,12000.0,2023-08-01,2023-08-01,Mexico,MXN,16.92,1500.0,88.652482,None,None,None,None


**Churn Report**

In [115]:
# Uploading to Google Sheets
print("Authenticating and opening spreadsheet...")
gc = pygsheets.authorize(custom_credentials=credentials)
print("Opening Spreadsheet..")
ss = gc.open_by_key("1nXLC0Ccu_hznyC8Ylci2BXlXnfwdv7GJFIVBc61I40U")

# Selecting location
ws = ss.worksheet("title","churn_raw")

# Uploading data
print("Clearing sheet ...")
ws.clear(start="A", end="AU")
print("Sheet cleared")

print("Uploading data...")
ws.set_dataframe(churn_sf_fx_2, start= "A1",copy_head= True, extend= True, nan=  "")
print("Data uploaded!")

Authenticating and opening spreadsheet...
Opening Spreadsheet..
Clearing sheet ...
Sheet cleared
Uploading data...
Data uploaded!


**List with all customers and IDs - ETL to G. Sheets**

Used to duplicate AM Customer Dash + Tutorial

In [116]:
# Create a df with org names and ids
clients_id = combined_data_fx[["country", "name", "org_id"]]

# Drop duplicates
clients_id = clients_id.drop_duplicates()

# Inspect
clients_id = clients_id.sort_values(by = "country")
clients_id.head()

,country,name,org_id
0,Brazil,Brownice - Tijuca - Doces e Sobremesas - Pik n...,e445cbbd-b698-43cf-9ccc-4362d07b4381
2770,Brazil,Sababa Doces Saudáveis,0a829f0c-cb14-4414-a441-7f5af299560d
2755,Brazil,Singello Brigadeiro,0699731f-f5aa-42d3-9307-69c76cea2575
2754,Brazil,Browneria do Astolfo - Pik n' Pak,8c6ae033-d22e-49c2-8f6e-89f4ef7b35d7
2735,Brazil,Roberta Lopes Confeitaria - Pik n' Pak,836818a8-2ab2-4f19-afa2-9fd43965beea


In [117]:
# Uploading to Google Sheets
print("Authenticating and opening spreadsheet...")
gc = pygsheets.authorize(custom_credentials=credentials)
print("Opening Spreadsheet..")
ss = gc.open_by_key("1ekw7gL7AOMnNWy_7T-rZNKM0erMVsCiUXJmuJcmvtAo")

# Selecting location
ws = ss.worksheet("title","clients_db")

# Uploading data
print("Clearing sheet ...")
ws.clear(start="A", end="Z")
print("Sheet cleared")

print("Uploading data...")
ws.set_dataframe(clients_id, start= "A1",copy_head= True, extend= True, nan=  "")
print("Data uploaded!")

Authenticating and opening spreadsheet...
Opening Spreadsheet..
Clearing sheet ...
Sheet cleared
Uploading data...
Data uploaded!
